<a href="https://colab.research.google.com/github/bhandrigan/bdh_colab_notebooks/blob/main/avs_billing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Load Packages
!pip install simple-salesforce XlsxWriter

import subprocess
import sys
import importlib
import google.cloud.bigquery
import google.cloud.storage
import google.oauth2
from google.cloud import bigquery, storage
from google.colab import auth, userdata
from google.oauth2 import service_account
import google.cloud.exceptions
from google.cloud.exceptions import NotFound
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from datetime import datetime, timedelta
import os
import json
import re
import uuid
import csv
import calendar
from calendar import week
import time
import random
import requests
import uuid
import simple_salesforce
from simple_salesforce import Salesforce, SalesforceMalformedRequest
import xlsxwriter


# Function to install and import modules and sub-modules
# def install_and_import(module_name, package_name=None, names=None, alias=None):
#     """
#     Attempts to import the module or specific names from the module.
#     If the module is not installed, it installs the package and then imports.

#     Parameters:
#     - module_name (str): The module to import.
#     - package_name (str): The pip package name to install if the module is not found.
#                           If None, assumes package name is the same as module name.
#     - names (list): List of specific names to import from the module.
#                     If None, imports the module itself.
#     - alias (str): Alias to assign to the imported module or name.
#     """
#     try:
#         if names:
#             module = importlib.import_module(module_name)
#             for name in names:
#                 imported_name = getattr(module, name)
#                 if alias and len(names) == 1:
#                     sys.modules[__name__].__dict__[alias] = imported_name
#                 else:
#                     sys.modules[__name__].__dict__[name] = imported_name
#             print(f"Imported {names} from {module_name}")
#         else:
#             imported_module = importlib.import_module(module_name)
#             if alias:
#                 sys.modules[__name__].__dict__[alias] = imported_module
#             else:
#                 sys.modules[__name__].__dict__[module_name] = imported_module
#             print(f"Imported {module_name}")
#     except ImportError:
#         if package_name is None:
#             package_name = module_name.replace('.', '-')
#         print(f"Installing package {package_name}")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
#         # Try importing again
#         try:
#             if names:
#                 module = importlib.import_module(module_name)
#                 for name in names:
#                     imported_name = getattr(module, name)
#                     if alias and len(names) == 1:
#                         sys.modules[__name__].__dict__[alias] = imported_name
#                     else:
#                         sys.modules[__name__].__dict__[name] = imported_name
#                 print(f"Installed {package_name} and imported {names} from {module_name}")
#             else:
#                 imported_module = importlib.import_module(module_name)
#                 if alias:
#                     sys.modules[__name__].__dict__[alias] = imported_module
#                 else:
#                     sys.modules[__name__].__dict__[module_name] = imported_module
#                 print(f"Installed {package_name} and imported {module_name}")
#         except ImportError as e:
#             print(f"Failed to import {module_name} after installing {package_name}: {e}")

# # Mapping of module names to pip package names
# module_to_pip = {
#     "google.cloud.bigquery": "google-cloud-bigquery",
#     "pandas_gbq": "pandas-gbq",
#     "simple_salesforce": "simple-salesforce",
#     "google.colab": "google-colab",
#     "google.oauth2": "google-auth",
#     "google.cloud.storage": "google-cloud-storage",
#     "google.cloud.exceptions": "google-cloud-exceptions",
#     "requests": "requests",
#     "xlsxwriter": "XlsxWriter",
#     "pandas": "pandas",
#     "numpy": "numpy",
#     "uuid": "uuid",
#     "re": None,
#     "os": None,
#     "json": None,
#     "datetime": None,
#     "time": None,
#     "random": None,
#     "csv": None,
#     "calendar": None,
# }

# # List of modules to check and import
# modules_to_import = [
#     {"module_name": "google.cloud", "names": ["bigquery", "storage"]},
#     {"module_name": "pandas_gbq", "names": None},
#     {"module_name": "simple_salesforce", "names": ["Salesforce", "SalesforceMalformedRequest"]},
#     {"module_name": "google.colab", "names": ["auth"]},
#     {"module_name": "google.oauth2", "names": ["service_account"]},
#     {"module_name": "google.cloud.storage", "names": None},
#     {"module_name": "google.cloud.exceptions", "names": ["NotFound"]},
#     {"module_name": "requests", "names": None},
#     {"module_name": "xlsxwriter", "names": None},
#     {"module_name": "pandas", "alias": "pd", "names": None},
#     {"module_name": "numpy", "alias": "np", "names": None},
#     {"module_name": "datetime", "names": ["datetime", "timedelta"]},
#     {"module_name": "os", "names": None},
#     {"module_name": "json", "names": None},
#     {"module_name": "re", "names": None},
#     {"module_name": "uuid", "names": None},
#     {"module_name": "csv", "names": None},
#     {"module_name": "time", "names": None},
#     {"module_name": "random", "names": None},
#     {"module_name": "calendar", "names": None},
#     {"module_name": "calendar", "names": ['week']},
# ]

# # Install and import modules
# for module_info in modules_to_import:
#     module_name = module_info.get('module_name')
#     package_name = module_to_pip.get(module_name)
#     names = module_info.get('names')
#     alias = module_info.get('alias')
#     install_and_import(module_name, package_name, names, alias)


print('Imported [\'userdata\'] from google.colab')
# Now you can use the imported modules and names directly
# For example, pandas is available as 'pd', numpy as 'np', etc.

# Example usage:
# df = pd.DataFrame()
# array = np.array([1, 2, 3])

In [ ]:
# @title Set Vars
# @markdown Enter the target billing month in ####.## format with #### being the year and ## being the broadcast month number from 01 to 12.
bcm_index_list_str = "2024.11" # @param {type:"string"}
bcm_index_list = [float(x) for x in bcm_index_list_str.split(",")]
bcm_index = bcm_index_list[-1]
# bcm_index = bcm_index_list[0]
# project_id = "adhoc-billing"
# @markdown Sync Salesforce with BVS before processing if this is the first time processing or data has changed and needs to be updated.
sync_bvs_with_sfdc_before_processing = True # @param {type:"boolean"}
sync_salesforce_with_bvs_before_processing = True # @param {type:"boolean"}
sync_salesforce_type = "FULL" # @param ["FULL","LIMITED","BILLING_RUN"]
limited_salesforce_objects = ["Task","Account","Product2","Pricebook2","PricebookEntry","Advertiser__c","Rate_Card__c","Encoder_Device__c","BVS_Format__c","BVS_Profile__c","BVS_Product_Code__c","BVS_Client_Code__c","BVS_Customer__c","Encoder_Group__c","Encoding_Advertiser__c","Encoding_Module_Code__c","Encoding_Product_Code__c"] # @param
billing_run_salesforce_objects = ["Task","Account","Opportunity","OpportunityLineItem","Advertiser__c","Rate_Card__c"] # @param
# @markdown Rebuild encodings if sfdc data or bvs data has changed that could affect ownership or alignment of encodings with advertisers.
rebuild_encodings = False # @param {type:"boolean"}
# @markdown Rebuild the detection archive if sfdc or bvs data has changed that could affect billing outcomes, you are changing billing months to assess or starting a new billing month.
rebuild_detection_archive = True # @param {type:"boolean"}
# @markdown Generate Encoder Excel Backup
gen_encoder_excel = False # @param {type:"boolean"}
# @markdown Generate Weekly Excel Backup
gen_weekly_excel = False # @param {type:"boolean"}
# @markdown Generate Monthly Invoice Excel Backup
gen_monthly_excel = True # @param {type:"boolean"}
# # @markdown Generate Monthly Usage Excel Backup
# generate_monthly_usage_excel = True # @param {type:"boolean"}
# # @markdown Generate Monthly Tiered Excel Backup
# generate_monthly_tiered_excel = True # @param {type:"boolean"}
# # @markdown Generate Monthly Detections Excel Backup
# generate_monthly_detections_excel = True # @param {type:"boolean"}
# @markdown Generate SFDC Tasks
generate_sfdc_tasks = False # @param {type:"boolean"}
# @markdown Process SFDC Task Responses
process_sfdc_tasks = True # @param {type:"boolean"}
# @markdown Send Opportunities to SFDC
send_opps_to_sfdc = True # @param {type:"boolean"}
process_sfdc_opps_from_invoices = True # @param {type:"boolean"}

test_mode = False # @param {type:"boolean"}
storage_bucket_name = 'advocado_billing_download' # @param {type:"string"}
project_id = 'adhoc-billing' # @param {type:"string"}
dataset_id = 'avs_billing_process' # @param {type:"string"}
encodings_matched_table_id = 'encodings_matched' # @param {type:"string"}
encodings_unmatched_table_id = 'encodings_unmatched' # @param {type:"string"}
encodings_matched_with_rates_table_id = 'encodings_matched_with_rates' # @param {type:"string"}
avs_encodings_master_table_id = 'avs_encodings_master' # @param {type:"string"}
billing_records_archive_table_id = 'billing_records_archive' # @param {type:"string"}
detections_detail_table_id = 'detections_detail' # @param {type:"string"}
billing_records__working_table_id = 'billing_records__working' # @param {type:"string"}
billing_invoice_files_table_id = '_billing_invoice_files' # @param {type:"string"}


sync_salesforce_auth_token = userdata.get('SYNC_SALESFORCE_API_KEY')
sync_salesforce_url = "https://us-central1-adhoc-billing.cloudfunctions.net/sync_salesforce" # @param {type:"string"}



In [ ]:
# @title Set SFDC Opportunity Vars
opp_owner_id = '0053h000006UDKhAAO' # @param {type:"string"}
opp_stage_name = 'Stage 3.5 - Billing Ready for Review' # @param {type:"string"}
opp_type = 'Existing Business' # @param {type:"string"}
opp_pricebook_id = '01s3h00000257iWAAQ' # @param {type:"string"}
opp_record_type_id = '0123h000000MtmPAAS' # @param {type:"string"}
sfdc_field_map = {}

In [ ]:
# @title create clients



USE_ENV_VARS = False
service_account_secret_name = 'SA_ADHOC_BILLING'
temp_credentials_location = "/tmp/credentials.json"

def get_service_account_credentials():
    try:
        if USE_ENV_VARS:
            key_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
            if key_path is None:
                raise EnvironmentError("Failed to find 'GOOGLE_APPLICATION_CREDENTIALS' in environment variables.")
            return service_account.Credentials.from_service_account_file(key_path)
        else:
            secret_value = userdata.get(service_account_secret_name)
            if not secret_value:
                raise ValueError("Failed to retrieve service account secret from Colab userdata.")
            credentials_dict = json.loads(secret_value)
            temp_credentials = temp_credentials_location
            with open(temp_credentials, "w") as f:
                json.dump(credentials_dict, f)
            os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = temp_credentials
            return service_account.Credentials.from_service_account_info(credentials_dict)
    except Exception as e:
        print(f"Error obtaining service account credentials: {e}")
        raise

# Create credentials from the service account info
credentials = get_service_account_credentials()
auth.authenticate_service_account()

# Initialize Google BigQuery Client
bigquery_client = bigquery.Client(credentials=credentials)

# Initialize Google Cloud Storage client
storage_client = storage.Client()

# Initialize Salesforce client
sf = Salesforce(username=userdata.get('SF_USERNAME'), password=userdata.get('SF_PASSWORD'), security_token=userdata.get('SF_TOKEN'))

# Get CAD to USD conversion rate

conversion_rate = None

def get_historical_cad_to_usd(date):
    api_key = '0fce96fb156d4f1eb391897e9d154e44'
    url = f'https://openexchangerates.org/api/historical/{date}.json?app_id={api_key}&symbols=CAD'
    response = requests.get(url)
    data = response.json()
    cad_to_usd = 1 / data['rates']['CAD']
    return cad_to_usd

def get_latest_exchange_rate_from_bq():
    query = """
    SELECT rate, date
    FROM `adhoc-billing.avs_billing_process.lu_usd_exchange_rates`
    WHERE currency = 'CAD'
    ORDER BY date DESC
    LIMIT 1
    """
    query_job = bigquery_client.query(query)
    results = query_job.result()
    if results.total_rows > 0:
        for row in results:
            return row.rate, row.date
    return None, None

def insert_exchange_rate_to_bq(date, rate):
    insert_query = f"""
    INSERT INTO `adhoc-billing.avs_billing_process.lu_usd_exchange_rates` (date, currency, rate)
    VALUES ('{date}', 'CAD', {rate})
    """
    bigquery_client.query(insert_query)

# Main logic
date_of_exchange = datetime.now().date()  # Get the current date as a datetime.date object
thirty_days_ago = date_of_exchange - timedelta(days=30)  # Subtract 30 days to get the comparison date

# Check for the most recent exchange rate in BigQuery
latest_rate, latest_date = get_latest_exchange_rate_from_bq()

if latest_date and latest_date >= thirty_days_ago:
    # Use the latest rate if it is within the last 30 days
    conversion_rate = latest_rate
    print(f"Using the latest CAD to USD conversion rate from BigQuery on {latest_date}: {conversion_rate}")
else:
    # Fetch the rate from the API and insert it into BigQuery
    conversion_rate = get_historical_cad_to_usd(date_of_exchange.strftime('%Y-%m-%d'))
    insert_exchange_rate_to_bq(date_of_exchange, conversion_rate)
    print(f"The CAD to USD conversion rate on {date_of_exchange} was {conversion_rate}, and it has been inserted into BigQuery.")


In [ ]:
# @title Sync Salesforce Function

def sync_salesforce_tables(salesforce_url, salesforce_auth_token, sync_type, salesforce_objects=[]):
    _sync_salesforce_url = salesforce_url
    _sync_salesforce_auth_token = salesforce_auth_token
    _sync_salesforce_type = sync_type
    _custom_salesforce_objects = salesforce_objects

    if sync_salesforce_with_bvs_before_processing:
        if _sync_salesforce_type == 'FULL':
            url = f"{_sync_salesforce_url}"
            params = {'authToken': _sync_salesforce_auth_token}

            response = requests.get(url, params=params)

            result = (f'Sync Salesforce FULL result: {response.status_code}. ' + f'Response: {response.text}')
            print(result)
            return result
        elif _sync_salesforce_type == 'LIMITED':
            data = {
                "authToken": _sync_salesforce_auth_token,
                "sfdc_objects_to_clone": limited_salesforce_objects
            }

            url = f"{_sync_salesforce_url}"
            headers = {'Content-Type': 'application/json'}

            response = requests.post(url, json=data, headers=headers)
            result = (f'Sync Salesforce LIMITED result: {response.status_code}. ' + f'Response: {response.text}')
            print(result)
            return result
        elif _sync_salesforce_type == 'BILLING_RUN':
            data = {
                "authToken": _sync_salesforce_auth_token,
                "sfdc_objects_to_clone": billing_run_salesforce_objects
            }

            url = f"{_sync_salesforce_url}"
            headers = {'Content-Type': 'application/json'}

            response = requests.post(url, json=data, headers=headers)

            result = (f'Sync Salesforce BILLING RUN result: {response.status_code}. ' + f'Response: {response.text}')
            print(result)
            return result
        elif _sync_salesforce_type == 'CUSTOM':
            data = {
                "authToken": _sync_salesforce_auth_token,
                "sfdc_objects_to_clone": _custom_salesforce_objects
            }

            url = f"{_sync_salesforce_url}"
            headers = {'Content-Type': 'application/json'}

            response = requests.post(url, json=data, headers=headers)

            result = (f'Sync Salesforce BILLING RUN result: {response.status_code}. ' + f'Response: {response.text}')
            print(result)
            return result

        else:
            print('Invalid sync_salesforce_type specified.')
            return 'Invalid sync_salesforce_type specified.'

    else:
        print('Skipping sync Salesforce')
        return 'Skipping sync Salesforce'




In [ ]:
# @title data fetch functions
# Define a function to fetch data from BigQuery
def fetch_gbq_data(query):
    try:
        _df = bigquery_client.query(query).to_dataframe()
        return _df
    except Exception as e:
        print(f"Error fetching data from GBQ: {e}")
        return pd.DataFrame()

# Define a function to fetch data from Salesforce
def fetch_sfdc_data(object_name, fields):
    try:
        query = f"SELECT Id, {', '.join(fields)} FROM {object_name}"
        result = sf.query_all(query)
        records = result['records']
        for record in records:
            record.pop('attributes', None)
        return records
        # return result['records']
    except Exception as e:
        print(f"Error fetching data from Salesforce: {e}")
        return []

def fetch_max_last_updated(object_name):
    try:
        query = f"SELECT Id, Last_Updated__c FROM {object_name} WHERE Last_Updated__c != null ORDER BY Last_Updated__c DESC LIMIT 1"
        result = sf.query(query)
        if result['totalSize'] > 0:
            record = result['records'][0]
            last_updated = record.get('Last_Updated__c')
            return last_updated
        else:
            return '1990-01-01 00:00:00'
            # return result['records']
    except Exception as e:
        print(f"Error fetching max last_updated from Salesforce: {e}")
        return None

In [ ]:
# @title General Utilities

# Function to clean the data
# look into how to improve
def clean_record(record):
    cleaned_record = {}
    for key, value in record.items():
        if pd.isna(value):
            cleaned_record[key] = None
        elif key == 'Year__c' and not str(value).isdigit():
            cleaned_record[key] = None
        else:
            cleaned_record[key] = value
    return cleaned_record



In [ ]:
#  @title SFDC Utilities

# Insert salesforce
def insert_salesforce(object_name, records, batch_size=500):
    """
    Insert records into Salesforce in batches with error handling and logging.

    Args:
    - object_name (str): The Salesforce object name.
    - records (list of dict): List of records to insert.
    - batch_size (int): Number of records to insert per batch. Default is 200.

    Returns:
    - None
    """
    total_records = len(records)
    print(f"Total records to insert: {total_records}")

    for i in range(0, total_records, batch_size):
        batch = records[i:i + batch_size]
        success_count = 0
        error_count = 0

        for record in batch:
            try:
                record = clean_record(record)
                sf.__getattr__(object_name).create(record)
                success_count += 1
            except SalesforceMalformedRequest as e:
                error_count += 1
                print(f"Error inserting record: {record}")
                print(f"Error message: {e.content}")
            except Exception as e:
                error_count += 1
                print(f"Unexpected error inserting record: {record}")
                print(f"Error message: {str(e)}")

        print(f"Batch {i//batch_size + 1}: Successfully inserted {success_count} records, {error_count} errors")

    print("Insert operation completed.")

def update_salesforce(object_name, records, batch_size=200):
    total_records = len(records)
    print(f"Total records to update: {total_records}")

    for i in range(0, total_records, batch_size):
        batch = records[i:i + batch_size]
        success_count = 0
        error_count = 0

        for record in batch:
            record_id = record.pop('Id')  # Remove the Id field from the record data
            record = clean_record(record)
            try:
                sf.__getattr__(object_name).update(record_id, record)
                success_count += 1
            except SalesforceMalformedRequest as e:
                error_count += 1
                print(f"Error updating record: {record}")
                print(f"Error message: {e.content}")
            except Exception as e:
                error_count += 1
                print(f"Unexpected error updating record: {record}")
                print(f"Error message: {str(e)}")

        print(f"Batch {i//batch_size + 1}: Successfully updated {success_count} records, {error_count} errors")

    print("Update operation completed.")

# Define a function to delete records where a specific field is empty or null
def delete_records_with_null_field(sf, object_name, field_name):
    query = f"SELECT Id FROM {object_name} WHERE {field_name} = null"
    result = sf.query_all(query)
    records = result['records']

    # Extract the Ids of the records to be deleted
    ids_to_delete = [record['Id'] for record in records]

    # Delete the records
    for record_id in ids_to_delete:
        try:
            sf.__getattr__(object_name).delete(record_id)
            print(f"Successfully deleted record with Id: {record_id}")
        except Exception as e:
            print(f"Error deleting record with Id: {record_id}")
            print(f"Error message: {e}")

# Define a function to sync data
def sync_data_ids(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field):
    merged_df = pd.DataFrame()
    object_to_sync = None
    # Fetch data from GBQ
    gbq_data = fetch_gbq_data(gbq_query)

    # Map GBQ fields to Salesforce fields
    gbq_data.rename(columns=gbq_to_sfdc_field_map, inplace=True)

    # Convert relevant fields to timestamps and fill NA with "1990-01-01 00:00:00"
    if sfdc_object_name != 'Task':
        for col in gbq_data.columns:
            if col.startswith('Last_Updated__c'):
                gbq_data[col] = pd.to_datetime(gbq_data[col], errors='coerce').fillna(pd.Timestamp('1990-01-01 00:00:00'))
                if gbq_data[col].dt.tz is None:  # Check if tz-naive
                    gbq_data[col] = gbq_data[col].dt.tz_localize('UTC')
                else:  # Convert tz-aware to UTC
                    gbq_data[col] = gbq_data[col].dt.tz_convert('UTC')

    # Ensure the external ID field is integer type
    gbq_data[sfdc_external_id_field] = gbq_data[sfdc_external_id_field].astype('Int64')

    print(f"GBQ Record Count: {len(gbq_data)}")
    sfdc_data = pd.DataFrame()

    # Fetch data from Salesforce
    sfdc_fields = list(gbq_to_sfdc_field_map.values())
    sfdc_raw = fetch_sfdc_data(sfdc_object_name, sfdc_fields)
    if sfdc_raw:
        sfdc_data = pd.DataFrame(sfdc_raw)
        print("SFDC Data (raw):")
    print(f"SFDC Record Count: {len(sfdc_data)}")

    # Ensure the external ID field is integer type in Salesforce data
    if len(sfdc_data) > 0:
        if sfdc_object_name != 'Task':
            sfdc_data[sfdc_external_id_field] = sfdc_data[sfdc_external_id_field].astype('Int64')
        else:
            sfdc_data[sfdc_external_id_field] = sfdc_data[sfdc_external_id_field].astype('Float64')

        # Convert relevant fields to timestamps and fill NA with "1990-01-01 00:00:00" in Salesforce data
        if sfdc_object_name != 'Task':
            for col in sfdc_data.columns:
                if col.startswith('Last_Updated__c'):
                    sfdc_data[col] = pd.to_datetime(sfdc_data[col], errors='coerce').fillna(pd.Timestamp('1990-01-01 00:00:00'))
                    if sfdc_data[col].dt.tz is None:  # Check if tz-naive
                        sfdc_data[col] = sfdc_data[col].dt.tz_localize('UTC')
                    else:  # Convert tz-aware to UTC
                        sfdc_data[col] = sfdc_data[col].dt.tz_convert('UTC')

        # Drop rows with NaN values in the external ID field
        sfdc_data.dropna(subset=[sfdc_external_id_field], inplace=True)

    if not sfdc_data.empty:
        print(f"SFDC Data to process: {len(sfdc_data)}")
    else:
        print('No SFDC records survived')

    # Merge GBQ data with Salesforce data on the external ID field
    if not sfdc_data.empty:
        merged_df = pd.merge(gbq_data, sfdc_data, how='left', on=sfdc_external_id_field, suffixes=('', '_sfdc'))

        # Filter records to insert (new records)
        new_records = merged_df[merged_df['Id'].isnull()].copy()
    else:
        new_records = gbq_data
        print(f"New Records to process: {len(new_records)}")

    # Filter records to update (existing records)
    existing_records = pd.DataFrame()
    if not merged_df.empty:
        if sfdc_object_name != 'Task':
            existing_records = merged_df[(merged_df['Last_Updated__c'] > merged_df['Last_Updated__c_sfdc']) & merged_df['Id'].notnull()].copy()
            print(f'Existing records to update: {len(existing_records)}')
        else:
            existing_records = merged_df[merged_df['Id'].notnull()].copy()
            print(f'Existing records to update: {len(existing_records)}')
    else:
        print('No existing records to update')

    # Prepare new records for insertion
    if not new_records.empty:
        if sfdc_object_name != 'Task':
            new_records['Last_Updated__c'] = new_records['Last_Updated__c'].dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
        new_records = new_records.drop(columns=[col for col in new_records.columns if col.endswith('_sfdc')] + ['Id'], errors='ignore')
        new_records = new_records.to_dict('records')
        print(f"New records to be added: {len(new_records)}")
        # Insert new records to Salesforce
        if new_records:
            insert_salesforce(sfdc_object_name, new_records)
            object_to_sync = sfdc_object_name
    else:
        new_records = []
        print('No new records to be added')

    # Update existing records in Salesforce
    if not existing_records.empty:
        if sfdc_object_name != 'Task':
            existing_records['Last_Updated__c'] = existing_records['Last_Updated__c'].dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
        existing_records = existing_records.drop(columns=[col for col in existing_records.columns if col.endswith('_sfdc')], errors='ignore')
        existing_records = existing_records.to_dict('records')
        print(f"Existing records to be updated: {len(existing_records)}")
        update_salesforce(sfdc_object_name, existing_records)
        object_to_sync = sfdc_object_name
    else:
        existing_records = []
        print('No existing records to update')

    return object_to_sync


# Define a function to sync data
def sync_data_ids_tasks(gbq_data, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field):
    merged_df = pd.DataFrame()

    # Map GBQ fields to Salesforce fields
    gbq_data.rename(columns=gbq_to_sfdc_field_map, inplace=True)

    # Ensure the external ID field is integer type

    gbq_data[sfdc_external_id_field] = gbq_data[sfdc_external_id_field].astype('Float64')
    # print("GBQ Data:")
    # print(gbq_data.head())
    print(f"GBQ Record Count: {len(gbq_data)}")
    sfdc_data = pd.DataFrame()
    # Fetch data from Salesforce
    sfdc_fields = list(gbq_to_sfdc_field_map.values())
    sfdc_raw = fetch_sfdc_data(sfdc_object_name, sfdc_fields)
    if sfdc_raw:
        sfdc_data = pd.DataFrame(sfdc_raw)
        print("SFDC Data (raw):")
    # print(sfdc_data.head())
    print(f"SFDC Record Count: {len(sfdc_data)}")

    # Ensure the external ID field is integer type in Salesforce data
    if len(sfdc_data) > 0:

        sfdc_data[sfdc_external_id_field] = sfdc_data[sfdc_external_id_field].astype('Float64')

        # Drop rows with NaN values in the external ID field
        sfdc_data.dropna(subset=[sfdc_external_id_field], inplace=True)
    if not sfdc_data.empty:
        print(f"SFDC Data to process: {len(sfdc_data)}")
        # print(sfdc_data.head())
    else:
        print('No SFDC records survived')

    # Merge GBQ data with Salesforce data on the external ID field
    if not sfdc_data.empty:
        merged_df = pd.merge(gbq_data, sfdc_data, how='left', on=sfdc_external_id_field, suffixes=('', '_sfdc'))

        # Filter records to insert (new records)
        new_records = merged_df[merged_df['Id'].isnull()].copy()
    else:
        new_records = gbq_data

        print(f"New Records to process: {len(new_records)}")
    # print(new_records.head())
    # print(new_records.dtypes)

    # Filter records to update (existing records)
    existing_records = pd.DataFrame()
    if not merged_df.empty:
        existing_records = merged_df[merged_df['Id'].notnull()].copy()
        print(f'Existing records to update: {len(existing_records)}')
    else:
        print('No existing records to update')

    # Prepare new records for insertion
    if not new_records.empty:
        new_records = new_records.drop(columns=[col for col in new_records.columns if col.endswith('_sfdc')] + ['Id'], errors='ignore')
        new_records = new_records.to_dict('records')
    else:
        new_records = []
        print('No new records to be added')

    # print(new_records)
    # Prepare existing records for update
    if not existing_records.empty:
        update_fields = list(gbq_to_sfdc_field_map.values())
        update_fields.remove(sfdc_external_id_field)  # External ID field should not be updated

        # Drop _sfdc suffix fields and prepare the final update records
        existing_records = existing_records[update_fields + ['Id']].drop(columns=[f'{col}_sfdc' for col in update_fields], errors='ignore')
        existing_records = existing_records.to_dict('records')
    else:
        existing_records = []  # Ensure existing_records is an empty list if no records to update

    print(f"Existing records to be updated: {len(existing_records)}" if existing_records else "No existing records to update")

    # Insert new records to Salesforce
    if new_records:
        insert_salesforce(sfdc_object_name, new_records)

    # Update existing records in Salesforce
    if existing_records:
        update_salesforce(sfdc_object_name, existing_records)

# continue from here
# Define a function to sync data
def sync_data_chars(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field):
    merged_df = pd.DataFrame()
    # Fetch data from GBQ
    gbq_data = fetch_gbq_data(gbq_query)

    # Map GBQ fields to Salesforce fields
    gbq_data.rename(columns=gbq_to_sfdc_field_map, inplace=True)

    # Convert relevant fields to timestamps and fill NA with "1990-01-01 00:00:00"
    for col in gbq_data.columns:
        if col.startswith('Last_Updated__c'):
            gbq_data[col] = pd.to_datetime(gbq_data[col], errors='coerce').fillna(pd.Timestamp('1990-01-01 00:00:00'))

    # Ensure the external ID field is integer type

    gbq_data[sfdc_external_id_field] = gbq_data[sfdc_external_id_field]
    print("GBQ Data:")
    print(gbq_data.head())

    # Fetch data from Salesforce
    sfdc_fields = list(gbq_to_sfdc_field_map.values())
    sfdc_raw = fetch_sfdc_data(sfdc_object_name, sfdc_fields)
    sfdc_data = pd.DataFrame(sfdc_raw)
    print("SFDC Data (raw):")
    print(sfdc_data.head())

    # Ensure the external ID field is integer type in Salesforce data
    sfdc_data[sfdc_external_id_field] = sfdc_data[sfdc_external_id_field].astype('Int64')

    # Convert relevant fields to timestamps and fill NA with "1990-01-01 00:00:00" in Salesforce data
    for col in sfdc_data.columns:
        if col.startswith('Last_Updated__c'):
            sfdc_data[col] = pd.to_datetime(sfdc_data[col], errors='coerce').fillna(pd.Timestamp('1990-01-01 00:00:00'))
            # Uncomment following line to force update
            # sfdc_data[col] = pd.Timestamp('1990-01-01 00:00:00')

    # Drop rows with NaN values in the external ID field
    sfdc_data.dropna(subset=[sfdc_external_id_field], inplace=True)
    if not sfdc_data.empty:
        print("SFDC Data (processed):")
        print(sfdc_data.head())
    else:
        print('No SFDC records survived')

    # Merge GBQ data with Salesforce data on the external ID field
    if not sfdc_data.empty:
        merged_df = pd.merge(gbq_data, sfdc_data, how='left', on=sfdc_external_id_field, suffixes=('', '_sfdc'))
        print("Merged Data:")
        print(merged_df.head())
        print(merged_df.dtypes)

        # Filter records to insert (new records)
        new_records = merged_df[merged_df['Id'].isnull()].copy()
    else:
        new_records = gbq_data

    print("New Records:")
    print(new_records.head())
    print(new_records.dtypes)

    # Filter records to update (existing records)
    existing_records = pd.DataFrame()
    if not merged_df.empty:
        existing_records = merged_df[(merged_df['Last_Updated__c'] > merged_df['Last_Updated__c_sfdc']) & merged_df['Id'].notnull()].copy()
        print(f'Existing records to update: {len(existing_records)}')

    # Prepare new records for insertion
    if not new_records.empty:
        new_records['Last_Updated__c'] = new_records['Last_Updated__c'].dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
        new_records = new_records.drop(columns=[col for col in new_records.columns if col.endswith('_sfdc')] + ['Id'], errors='ignore')
        new_records = new_records.to_dict('records')
        print(f"New records to be added: {len(new_records)}")
    else:
        new_records = []

    print(new_records)
    # Prepare existing records for update
    if not existing_records.empty:
        update_fields = list(gbq_to_sfdc_field_map.values())
        update_fields.remove(sfdc_external_id_field)  # External ID field should not be updated

        # Convert Last_Updated__c field to the desired string format
        existing_records['Last_Updated__c'] = existing_records['Last_Updated__c'].dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]

        # Drop _sfdc suffix fields and prepare the final update records
        existing_records = existing_records[update_fields + ['Id']].drop(columns=[f'{col}_sfdc' for col in update_fields], errors='ignore')
        existing_records = existing_records.to_dict('records')
    else:
        existing_records = []  # Ensure existing_records is an empty list if no records to update

    print(f"Existing records to be updated: {len(existing_records)}" if existing_records else "No existing records to update")

    # Insert new records to Salesforce
    if new_records:
        insert_salesforce(sfdc_object_name, new_records)

    # Update existing records in Salesforce
    if existing_records:
        update_salesforce(sfdc_object_name, existing_records)

def update_bvs_customers_lookup_on_format(sf):
    # Query all BVS_Formats__c records without the BVS_Customer__c lookup field set
    query = "SELECT Id, Customer_ID__c FROM BVS_Format__c WHERE BVS_Customer__c = null"
    bvs_format_records = sf.query_all(query)
    print(len(bvs_format_records))

    for record in bvs_format_records['records']:
        bvs_format_record_id = record['Id']
        customer_id = record['Customer_ID__c']

        # Search for the BVS_Customer__c record with matching customer_id__c
        customer_query = f"SELECT Id FROM BVS_Customer__c WHERE customer_id__c = {customer_id}"
        result = sf.query(customer_query)

        if result['totalSize'] == 1:
            # If a matching BVS_Customer__c record is found, get its Id
            bvs_customer_id = result['records'][0]['Id']

            # Update the BVS_Formats__c record with the BVS_Customer lookup field
            sf.BVS_Format__c.update(bvs_format_record_id, {'BVS_Customer__c': bvs_customer_id})
            print(f"Updated BVS_Formats__c record {bvs_format_record_id} with BVS_Customer__c {bvs_customer_id}")
        else:
            # Handle case where no matching or multiple records are found
            print(f"No matching or multiple BVS_Customer__c records found for Customer_ID__c: {customer_id}")

def update_bvs_profile_lookup_on_format(sf):
    # Query all BVS_Formats__c records without the BVS_Customer__c lookup field set
    query = "SELECT Id, Profile_ID__c FROM BVS_Format__c WHERE BVS_Profile__c = null"
    bvs_format_records = sf.query_all(query)
    print(len(bvs_format_records))

    for record in bvs_format_records['records']:
        bvs_format_record_id = record['Id']
        profile_id = record['Profile_ID__c']

        # Search for the BVS_Profile__c record with matching BVS_Profile_ID__c
        profile_query = f"SELECT Id FROM BVS_Profile__c WHERE BVS_Profile_ID__c = {profile_id}"
        result = sf.query(profile_query)

        if result['totalSize'] == 1:
            # If a matching BVS_Profile__c record is found, get its Id
            bvs_profile_id = result['records'][0]['Id']

            # Update the BVS_Formats__c record with the BVS_Profile lookup field
            sf.BVS_Format__c.update(bvs_format_record_id, {'BVS_Profile__c': bvs_profile_id})
            print(f"Updated BVS_Formats__c record {bvs_format_record_id} with BVS_Profile__c {bvs_profile_id}")
        else:
            # Handle case where no matching or multiple records are found
            print(f"No matching or multiple BVS_Profile__c records found for BVS_Profile_ID__c: {profile_id}")

def sync_data_with_precalculated_id(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field):
    # Fetch data from GBQ
    object_to_sync = None
    gbq_data = fetch_gbq_data(gbq_query)

    # Map GBQ fields to Salesforce fields
    gbq_data.rename(columns=gbq_to_sfdc_field_map, inplace=True)

    # print("GBQ Data:")
    # print(gbq_data.head())

    # Fetch data from Salesforce
    sfdc_fields = list(gbq_to_sfdc_field_map.values())
    sfdc_raw = fetch_sfdc_data(sfdc_object_name, sfdc_fields)
    sfdc_data = pd.DataFrame(sfdc_raw)
    # print("SFDC Data (raw):")
    # print(sfdc_data.head())

    # Drop rows with NaN values in the external ID field in Salesforce data if data is not empty
    if not sfdc_data.empty:
        sfdc_data.dropna(subset=[sfdc_external_id_field], inplace=True)
        print("SFDC Data (processed):")
        # print(sfdc_data.head())
    else:
        print('No SFDC records survived')

    # Merge GBQ data with Salesforce data on the external ID field
    if not sfdc_data.empty:
        merged_df = pd.merge(gbq_data, sfdc_data, how='left', on=sfdc_external_id_field, suffixes=('', '_sfdc'))
        print("Merged Data:")
        # print(merged_df.head())
        # print(merged_df.dtypes)

        # Filter records to insert (new records)
        new_records = merged_df[merged_df['Id'].isnull()].copy()
    else:
        new_records = gbq_data

    # print("New Records:")
    # print(new_records.head())
    # print(new_records.dtypes)

    # Filter records to update (existing records)
    existing_records = pd.DataFrame()
    if not sfdc_data.empty and not merged_df.empty:
        existing_records = merged_df[merged_df['Id'].notnull()].copy()
        print(f'Existing records to update: {len(existing_records)}')

    # Prepare new records for insertion
    if not new_records.empty:
        new_records = new_records.drop(columns=[col for col in new_records.columns if col.endswith('_sfdc')] + ['Id'], errors='ignore')
        new_records = new_records.to_dict('records')
        print(f"New records to be added: {len(new_records)}")
    else:
        new_records = []

    print(new_records)
    # Prepare existing records for update
    if not existing_records.empty:
        update_fields = list(gbq_to_sfdc_field_map.values())
        update_fields.remove(sfdc_external_id_field)  # External ID field should not be updated

        # Drop _sfdc suffix fields and prepare the final update records
        existing_records = existing_records[update_fields + ['Id']].drop(columns=[f'{col}_sfdc' for col in update_fields], errors='ignore')
        existing_records = existing_records.to_dict('records')
    else:
        existing_records = []  # Ensure existing_records is an empty list if no records to update

    print(f"Existing records to be updated: {len(existing_records)}" if existing_records else "No existing records to update")

    # Insert new records to Salesforce
    if new_records:
        insert_salesforce(sfdc_object_name, new_records)
        object_to_sync = sfdc_object_name

    # Update existing records in Salesforce
    if existing_records:
        update_salesforce(sfdc_object_name, existing_records)
        object_to_sync = sfdc_object_name
    return object_to_sync

SHORTIO_API_KEY = userdata.get('SHORTIO_API_KEY')

def get_short_url(long_url):
    res = requests.post('https://api.short.io/links', json={
        'domain': 'link.veil.global',
        'originalURL': long_url,
    }, headers = {
        'authorization': SHORTIO_API_KEY,
        'content-type': 'application/json'
    }, )

    res.raise_for_status()
    data = res.json()
    short_url = data['shortURL']
    return short_url

# # Update the BigQuery table
# def update_bq_invoice_table_short_url(df, table_id):
#     for index, row in df.iterrows():
#         query = f"""
#         UPDATE `{table_id}`
#         SET excel_path_short = '{row['short_url']}'
#         WHERE excel_path = '{row['long_url']}'
#         AND excel_path_short IS NULL
#         """
#         query_job = bigquery_client.query(query)
#         query_job.result()  # Wait for the query to complete

def update_bq_invoice_table_short_url(df, table_id):

    # Step 1: Write the DataFrame to a temporary table in BigQuery
    temp_table_name = f"temp_table_{uuid.uuid4().hex}"
    temp_table_id = f"{project_id}.{dataset_id}.{temp_table_name}"

    # Ensure the temporary dataset exists
    dataset_ref = bigquery.DatasetReference(bigquery_client.project, dataset_id)
    try:
        bigquery_client.get_dataset(dataset_ref)
    except Exception:
        dataset = bigquery.Dataset(dataset_ref)
        bigquery_client.create_dataset(dataset)

    # Load DataFrame to the temporary table
    job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
    load_job = bigquery_client.load_table_from_dataframe(df, temp_table_id, job_config=job_config)
    load_job.result()  # Wait for the load job to complete

    # Step 2: Perform the batch update using a single query
    query = f"""
    UPDATE `{table_id}` AS T
    SET excel_path_short = S.short_url
    FROM `{temp_table_id}` AS S
    WHERE T.excel_path = S.long_url
    AND T.excel_path_short IS NULL
    """
    query_job = bigquery_client.query(query)
    query_job.result()  # Wait for the query to complete

    # Step 3: Delete the temporary table
    bigquery_client.delete_table(temp_table_id)

def update_bq_sfdc_opportunity_id(df, table_id):
    dataset_id = 'avs_billing_process'
    # Step 1: Write the DataFrame to a temporary table in BigQuery
    temp_table_name = f"temp_table_{uuid.uuid4().hex}"
    temp_table_id = f"{project_id}.{dataset_id}.{temp_table_name}"

    # Ensure the temporary dataset exists
    dataset_ref = bigquery.DatasetReference(bigquery_client.project, dataset_id)
    try:
        bigquery_client.get_dataset(dataset_ref)
    except Exception:
        dataset = bigquery.Dataset(dataset_ref)
        bigquery_client.create_dataset(dataset)

    # Load DataFrame to the temporary table
    job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
    load_job = bigquery_client.load_table_from_dataframe(df, temp_table_id, job_config=job_config)
    load_job.result()  # Wait for the load job to complete

    # Step 2: Perform the batch update using a single query
    query = f"""
    UPDATE `{table_id}` AS T
    SET sfdc_opportunity_id = S.sfdc_opportunity_id
    FROM `{temp_table_id}` AS S
    WHERE T.invoice_id = S.invoice_id
    """
    query_job = bigquery_client.query(query)
    query_job.result()  # Wait for the query to complete

    # Step 3: Delete the temporary table
    bigquery_client.delete_table(temp_table_id)



In [ ]:
# @title Billing Process Functions

def update_encodings_base():
    query = f"""


    DROP TABLE IF EXISTS `{project_id}.{dataset_id}.{encodings_matched_table_id}`;
    CREATE TABLE `{project_id}.{dataset_id}.{encodings_matched_table_id}` AS

    (
        WITH baseEncodings AS (
        SELECT distinct
        e.encoding_id, e.format_id, e.encoder_group_id, e.encoded_timestamp,
        e.clone_of, e.status, e.last_updated, e.last_audit_id, e.encoder_id, e.detection_end_date
        , f.format_name,
        -- f.customer_id,
        f.profile_id, c.customer_name, p.profile_name,
        CASE WHEN (e.attributes.isci is NOT NULL) THEN e.attributes.isci
        WHEN (e.attributes.isci is NULL AND e.attributes.project_name is not null) THEN e.attributes.project_name
        WHEN (e.attributes.isci is NULL AND e.attributes.project_name is null and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description not like 'TV%' or e.attributes.description not like 'RA%')) THEN trim(substring(e.attributes.description, 9,10))
        WHEN (e.attributes.isci is NULL AND e.attributes.project_name is null and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description  like 'TV%' or e.attributes.description  like 'RA%')) THEN trim(substring(e.attributes.description, 19,20))
        ELSE NULL END AS isci,

        CASE WHEN (e.attributes.product_code is NOT NULL) THEN e.attributes.product_code
        WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NOT NULL) THEN e.attributes.product_name
        WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NOT NULL) THEN e.attributes.donovan_agency_product_code
        WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description not like 'TV%' or e.attributes.description not like 'RA%')) THEN trim(substring(e.attributes.description, 27,4))
        WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description  like 'TV%' or e.attributes.description  like 'RA%')) THEN trim(substring(e.attributes.description, 7,4))
        ELSE NULL END AS product_code,

        CASE WHEN (e.attributes.advertiser is NOT NULL) THEN e.attributes.advertiser
        WHEN (e.attributes.advertiser is NULL AND e.attributes.client_code is NOT NULL) THEN e.attributes.client_code
        WHEN (e.attributes.advertiser is NULL AND e.attributes.client_code is NULL AND e.attributes.donovan_agency_advertiser_code is NOT NULL) THEN e.attributes.donovan_agency_advertiser_code
        WHEN (e.attributes.advertiser is NULL AND e.attributes.client_code is NULL AND e.attributes.donovan_agency_advertiser_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description not like 'TV%' or e.attributes.description not like 'RA%')) THEN trim(substring(e.attributes.description, 23,4))
        WHEN (e.attributes.advertiser is NULL AND e.attributes.client_code is NULL AND e.attributes.donovan_agency_advertiser_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description  like 'TV%' or e.attributes.description  like 'RA%')) THEN trim(substring(e.attributes.description, 3,4))
        ELSE NULL END AS advertiser,



        sc.Account__c AS sfdc_account_id,
        acct.Name AS sfdc_account_name,
        f.deleted as format_deleted, c.deleted as customer_deleted, p.deleted as profile_deleted
        FROM `{project_id}.{dataset_id}.avs_encodings_master` e
        LEFT JOIN `bigquery-sandbox-393916.prod_avs.formats` f
        USING (format_id)
        LEFT JOIN `bigquery-sandbox-393916.prod_avs.customers` c
        USING (customer_id)
        LEFT JOIN `bigquery-sandbox-393916.prod_avs.profiles` p
        USING (profile_id)
        LEFT JOIN `{project_id}.{dataset_id}.sfdc_bvs_customer__c_obj` sc
        ON c.customer_id = sc.customer_id__c
        LEFT JOIN `{project_id}.{dataset_id}.sfdc_account_obj` acct
        ON sc.Account__c = acct.Id
        )

        select be.*,
            cast(null as string) AS sfdc_advertiser_id,
            cast(null as string) as sfdc_advertiser_name,
            cast(null as string) as sfdc_advertiser_match_type,
            cast(null as string) AS sfdc_rate_card_id,
            cast(null as string) AS ad_prod_campaign,
            -- cast(null as string) AS advertiser_status,
            -- cast(null as string) AS billing_frequency,
            -- cast(null as string) AS invoice_format,
            cast(null as timestamp) AS advertiser_updated,
            e.attributes.spot_estimate as spot_estimate,
            e.attributes.cable_estimate as cable_estimate,
            e.attributes.campaign as campaign,


        # e.attributes
        from baseEncodings be
        join `{project_id}.{dataset_id}.{avs_encodings_master_table_id}` e
        USING (encoding_id)
    );

    UPDATE `{project_id}.{dataset_id}.{encodings_matched_table_id}` em
        set ad_prod_campaign = replace( (concat(trim((advertiser)),'-',trim((product_code)), '-',trim((campaign)))),' ','_')
        where ad_prod_campaign is null;

    UPDATE `{project_id}.{dataset_id}.{encodings_matched_table_id}` em
        SET em.sfdc_advertiser_id = adv.Id,
        em.sfdc_advertiser_name = adv.Name,
        em.sfdc_advertiser_match_type = adv.Match_Type__c,
        em.sfdc_rate_card_id = adv.Related_Rate_Card__c,
        -- em.advertiser_status = adv.Status__c,
        -- em.billing_frequency = adv.Billing_Frequency__c,
        -- em.invoice_format = adv.Invoice_Format__c,
        em.advertiser_updated = current_timestamp()
        FROM `{project_id}.{dataset_id}.sfdc_advertiser__c_obj` adv,
        UNNEST(SPLIT(adv.Product_Code__c, ',')) AS product_code
        WHERE em.product_code = product_code
        -- AND em.sfdc_account_id = adv.Account__c
        AND adv.Encoding_Format_ID__c is NOT NULL
        AND em.format_id = adv.Encoding_Format_ID__c
        AND adv.Match_Type__c = 'encoding_product_code_multiple'
        AND  coalesce(em.sfdc_advertiser_id, '') = ''
        ;

        UPDATE `{project_id}.{dataset_id}.{encodings_matched_table_id}` em
        SET em.sfdc_advertiser_id = adv.Id,
        em.sfdc_advertiser_name = adv.Name,
        em.sfdc_advertiser_match_type = adv.Match_Type__c,
        em.sfdc_rate_card_id = adv.Related_Rate_Card__c,
        -- em.advertiser_status = adv.Status__c,
        -- em.billing_frequency = adv.Billing_Frequency__c,
        -- em.invoice_format = adv.Invoice_Format__c,
        em.advertiser_updated = current_timestamp()
        FROM `{project_id}.{dataset_id}.sfdc_advertiser__c_obj` adv
        WHERE em.product_code = adv.Enc_Product_Code__c
        -- AND em.sfdc_account_id = adv.Account__c
        AND adv.Encoding_Format_ID__c is NOT NULL
        AND em.format_id = adv.Encoding_Format_ID__c
        AND adv.Match_Type__c = 'encoding_product_code'
        AND  coalesce(em.sfdc_advertiser_id, '') = ''
        ;

        UPDATE `{project_id}.{dataset_id}.{encodings_matched_table_id}` em
        SET em.sfdc_advertiser_id = adv.Id,
        em.sfdc_advertiser_name = adv.Name,
        em.sfdc_advertiser_match_type = adv.Match_Type__c,
        em.sfdc_rate_card_id = adv.Related_Rate_Card__c,
        -- em.advertiser_status = adv.Status__c,
        -- em.billing_frequency = adv.Billing_Frequency__c,
        -- em.invoice_format = adv.Invoice_Format__c,
        em.advertiser_updated = current_timestamp()
        FROM `{project_id}.{dataset_id}.sfdc_advertiser__c_obj` adv
        WHERE em.product_code = adv.Enc_Product_Code__c
        -- AND em.sfdc_account_id = adv.Account__c
        AND adv.Match_Type__c = 'encoding_product_code_ignore_format'
        AND  coalesce(em.sfdc_advertiser_id, '') = ''
        ;

        UPDATE `{project_id}.{dataset_id}.{encodings_matched_table_id}` em
        SET em.sfdc_advertiser_id = adv.Id,
        em.sfdc_advertiser_name = adv.Name,
        em.sfdc_advertiser_match_type = adv.Match_Type__c,
        em.sfdc_rate_card_id = adv.Related_Rate_Card__c,
        -- em.advertiser_status = adv.Status__c,
        -- em.billing_frequency = adv.Billing_Frequency__c,
        -- em.invoice_format = adv.Invoice_Format__c,
        em.advertiser_updated = current_timestamp()
        FROM `{project_id}.{dataset_id}.sfdc_advertiser__c_obj` adv
        WHERE em.advertiser = adv.Enc_Advertiser__c
        -- AND em.sfdc_account_id = adv.Account__c
        AND adv.Encoding_Format_ID__c is NOT NULL
        AND em.format_id = adv.Encoding_Format_ID__c
        AND adv.Match_Type__c = 'encoding_advertiser'
        AND  coalesce(em.sfdc_advertiser_id, '') = ''
        ;

        UPDATE `{project_id}.{dataset_id}.{encodings_matched_table_id}` em
        SET em.sfdc_advertiser_id = adv.Id,
        em.sfdc_advertiser_name = adv.Name,
        em.sfdc_advertiser_match_type = adv.Match_Type__c,
        em.sfdc_rate_card_id = adv.Related_Rate_Card__c,
        -- em.advertiser_status = adv.Status__c,
        -- em.billing_frequency = adv.Billing_Frequency__c,
        -- em.invoice_format = adv.Invoice_Format__c,
        em.advertiser_updated = current_timestamp()
        FROM `{project_id}.{dataset_id}.sfdc_advertiser__c_obj` adv
        WHERE em.advertiser = adv.Enc_Advertiser__c
        -- AND em.sfdc_account_id = adv.Account__c
        AND adv.Match_Type__c = 'encoding_advertiser_ignore_format'
        AND  coalesce(em.sfdc_advertiser_id, '') = ''
        ;

        UPDATE `{project_id}.{dataset_id}.{encodings_matched_table_id}` em
        SET em.sfdc_advertiser_id = adv.Id,
        em.sfdc_advertiser_name = adv.Name,
        em.sfdc_advertiser_match_type = adv.Match_Type__c,
        em.sfdc_rate_card_id = adv.Related_Rate_Card__c,
        -- em.advertiser_status = adv.Status__c,
        -- em.billing_frequency = adv.Billing_Frequency__c,
        -- em.invoice_format = adv.Invoice_Format__c,
        em.advertiser_updated = current_timestamp()
        FROM `{project_id}.{dataset_id}.sfdc_advertiser__c_obj` adv
        WHERE
        -- em.sfdc_account_id = adv.Account__c
        -- AND
        em.format_id  = adv.Encoding_Format_ID__c
        AND adv.Match_Type__c = 'encoding_format'
        AND  coalesce(em.sfdc_advertiser_id, '') = ''
        ;
        DROP TABLE IF EXISTS `{project_id}.{dataset_id}.{encodings_matched_with_rates_table_id}`;
        CREATE TABLE `{project_id}.{dataset_id}.{encodings_matched_with_rates_table_id}` AS (
        select * from `{project_id}.{dataset_id}.{encodings_matched_table_id}`
        left join `{project_id}.{dataset_id}.sfdc_advertiser_rate_card_prepped`
        using (sfdc_account_id, sfdc_advertiser_id, sfdc_rate_card_id)

        )

    """

    # Execute the query
    query_job = bigquery_client.query(query)

    # Wait for the query to finish
    query_job.result()

    print(f"Encodings have been rebuilt")

def get_unmatched_advertisers():
    query = f"""
    select * FROM `{project_id}.{dataset_id}.billing_unmatched_advertisers_formats`
    """

    # Execute the query
    query_job = bigquery_client.query(query)

    # Wait for the query to finish encodings_matched_with_rates
    query_job.result()

    # Get the results as a DataFrame
    df2 = query_job.to_dataframe()

    # Display the DataFrame
    print(f"Unmatched Advertisers: {len(df2)}")
    print(df2.head(len(df2)))
    return

def get_unmatched_accounts():
    query = f"""
        DECLARE missing_customer_ids ARRAY<INT64>;
        DECLARE missing_format_ids ARRAY<INT64>;
        DECLARE null_string STRING DEFAULT NULL;
        DECLARE null_int INT64 DEFAULT NULL;
        DECLARE null_datetime DATETIME DEFAULT NULL;

        SET missing_customer_ids = (
        SELECT ARRAY_AGG(customer_id)
        FROM (
            SELECT customer_id
            FROM `{project_id}.{dataset_id}.sfdc_bvs_customer__c_obj` AS sfdc
            RIGHT JOIN `bigquery-sandbox-393916.prod_avs.customers` AS prod
            ON sfdc.customer_id__c = prod.customer_id
            WHERE sfdc.customer_id__c IS NULL
        )
        );

        select * from `bigquery-sandbox-393916.prod_avs.customers`
        where customer_id in (SELECT *
        FROM UNNEST(missing_customer_ids));

        SET missing_format_ids = (
        SELECT ARRAY_AGG(format_id)
        FROM (
            SELECT format_id
            FROM `bigquery-sandbox-393916.prod_avs.formats`
            WHERE customer_id IN (SELECT *
        FROM UNNEST(missing_customer_ids))
        )
        );

        DROP TABLE IF EXISTS `{project_id}.{dataset_id}.{encodings_unmatched_table_id}`;
        CREATE TABLE `{project_id}.{dataset_id}.{encodings_unmatched_table_id}` AS
        (
        SELECT e.*, f.format_name, f.customer_id, f.profile_id, c.customer_name, p.profile_name, null_string AS sfdc_account_id, null_string AS sfdc_account_name , null_string AS sfdc_advertiser_id, null_string as sfdc_advertiser_name, null_string as sfdc_advertiser_match_type, null_string AS sfdc_rate_card_id, null_string AS advertiser_status, null_string AS billing_frequency, null_string AS invoice_format, null_datetime AS last_updated2
        FROM `bigquery-sandbox-393916.prod_avs.encodings` e
        JOIN `bigquery-sandbox-393916.prod_avs.formats` f
        USING (format_id)
        JOIN `bigquery-sandbox-393916.prod_avs.customers` c
        USING (customer_id)
        JOIN `bigquery-sandbox-393916.prod_avs.profiles` p
        USING (profile_id)
        WHERE format_id in (SELECT * FROM UNNEST(missing_format_ids)))

        ;


    """

    # Execute the query
    query_job = bigquery_client.query(query)

    query = f"""
    select * FROM `{project_id}.{dataset_id}.{encodings_unmatched_table_id}`
    """

    # Execute the query
    query_job = bigquery_client.query(query)

    # Wait for the query to finish
    query_job.result()

    # Get the results as a DataFrame
    df3 = query_job.to_dataframe()

    # Display the DataFrame
    print(f"Unmatched Accounts: {len(df3)}")
    print(df3.head(len(df3)))
    return

def load_detections_into_archive(bcm_index):
    query = f"""
        DECLARE bcm_value FLOAT64 DEFAULT {bcm_index};

        DECLARE rebuild_archives BOOL DEFAULT {rebuild_detection_archive};


        DELETE FROM `{project_id}.{dataset_id}.{billing_records_archive_table_id}`
        WHERE bcm_index = bcm_value;
        -- DROP TABLE IF EXISTS `{project_id}.{dataset_id}.{billing_records_archive_table_id}`;
        INSERT INTO `{project_id}.{dataset_id}.{billing_records_archive_table_id}`
        -- CREATE TABLE `{project_id}.{dataset_id}.{billing_records_archive_table_id}` AS
        (select *
        from `{project_id}.{dataset_id}.{detections_detail_table_id}`
        join `{project_id}.{dataset_id}.{encodings_matched_with_rates_table_id}`
        on detection_encoding_id = encoding_id
        WHERE
        bcm_index = bcm_value)
        ;


    """

    # Execute the query
    query_job = bigquery_client.query(query)

    # Wait for the query to finish
    query_job.result()

    query = f"""
        DROP TABLE IF EXISTS `{project_id}.{dataset_id}.{billing_records__working_table_id}`;

        CREATE TABLE `{project_id}.{dataset_id}.{billing_records__working_table_id}` AS
        (
        SELECT * FROM `{project_id}.{dataset_id}.{billing_records_archive_table_id}`
        WHERE
        bcm_index = {bcm_index});
        """
    # Execute the query
    query_job = bigquery_client.query(query)

    # Wait for the query to finish
    query_job.result()
    print(f"Billing records have been archived")

    query = f"""
        DECLARE bcm_value FLOAT64 DEFAULT {bcm_index};

        DECLARE rebuild_archives BOOL DEFAULT {rebuild_detection_archive};


        DELETE FROM `{project_id}.{dataset_id}.__detections_billing_summary`
        WHERE bcm_index = bcm_value;
        -- DROP TABLE IF EXISTS `{project_id}.{dataset_id}.{billing_records_archive_table_id}`;
        INSERT INTO `{project_id}.{dataset_id}.__detections_billing_summary`
        -- CREATE TABLE `{project_id}.{dataset_id}.{billing_records_archive_table_id}` AS
        # (CREATE TABLE `adhoc-billing.avs_billing_process.__detections_billing_summary` AS
        # (
        (with modusAndChief AS (
        select encoding_id as detection_encoding_id, sfdc_account_name, sfdc_advertiser_name, format_name from `adhoc-billing.avs_billing_process.encodings_matched`
        where format_deleted is FALSE
        and customer_deleted is FALSE
        and profile_deleted is FALSE
        ),
        directvDish as (select broadcaster_id from `bigquery-sandbox-393916.mongo.master_channels` where dma_id in (select dma_id from `bigquery-sandbox-393916.mongo.dmas` where name in ('DIRECTV', 'DISH'))),

        totals AS (
        select bcm_index, sfdc_account_name, sfdc_advertiser_name, format_name, count(occurrence_id) as gross_detections,
        case when (any_value(d.broadcaster_id) is null) then FALSE ELSE TRUE END AS is_directv_dish,
        from `adhoc-billing.avs_billing_process.detections_detail`
        join modusAndChief using (detection_encoding_id)
        left join directvDish d using (broadcaster_id)
        group by bcm_index, sfdc_account_name, sfdc_advertiser_name, format_name
        order by bcm_index, sfdc_account_name, sfdc_advertiser_name, format_name
        )
        select *
        , sum(gross_detections) over (partition by sfdc_advertiser_name) as advertiser_total_gross_detections,
        sum(gross_detections) over (partition by format_name) as format_total_gross_detections,
        sum(gross_detections) over (partition by sfdc_account_name) as account_total_gross_detections,
        sum(case when (is_directv_dish is FALSE) then gross_detections else 0 end) over (partition by sfdc_advertiser_name) as advertiser_total_gross_detections_no_directv_dish,
        sum(case when (is_directv_dish is FALSE) then gross_detections else 0 end) over (partition by format_name) as format_total_gross_detections_no_directv_dish,
        sum(case when (is_directv_dish is FALSE) then gross_detections else 0 end) over (partition by sfdc_account_name) as account_total_gross_detections_no_directv_dish
        from totals
        where bcm_index = bcm_value
        )
        ;


    """

    # Execute the query
    query_job = bigquery_client.query(query)

    # Wait for the query to finish
    query_job.result()

# Function to generate a signed URL for GCS object
def generate_signed_url(bucket_name, blob_name, expiration=30):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    url = blob.generate_signed_url(
        expiration=datetime.utcnow() + timedelta(days=expiration),
        method="GET"
    )
    return url

def upsert_invoice_record(invoice_id, file_url, bcm_index, sfdc_account_id, sfdc_advertiser_id, billing_type):
    table_id = f"{project_id}.{dataset_id}.{billing_invoice_files_table_id}"

    # Define the SQL query for the upsert (MERGE)
    query = f"""
    MERGE `{table_id}` AS target
    USING (
        SELECT
            @invoice_id AS invoice_id,
            @file_url AS excel_path,
            @bcm_index AS bcm_index,
            @sfdc_account_id AS sfdc_account_id,
            @sfdc_advertiser_id AS sfdc_advertiser_id,
            @billing_type AS billing_type
    ) AS source
    ON target.invoice_id = source.invoice_id
    WHEN MATCHED THEN
        UPDATE SET
            excel_path = source.excel_path,
            bcm_index = source.bcm_index,
            sfdc_account_id = source.sfdc_account_id,
            sfdc_advertiser_id = source.sfdc_advertiser_id,
            billing_type = source.billing_type
    WHEN NOT MATCHED THEN
        INSERT (invoice_id, excel_path, bcm_index, sfdc_account_id, sfdc_advertiser_id, billing_type)
        VALUES (source.invoice_id, source.excel_path, source.bcm_index, source.sfdc_account_id, source.sfdc_advertiser_id, source.billing_type)
    """

    # Define the query parameters
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("invoice_id", "STRING", str(invoice_id)),
            bigquery.ScalarQueryParameter("file_url", "STRING", file_url),
            bigquery.ScalarQueryParameter("bcm_index", "STRING", bcm_index),
            bigquery.ScalarQueryParameter("sfdc_account_id", "STRING", sfdc_account_id),
            bigquery.ScalarQueryParameter("sfdc_advertiser_id", "STRING", sfdc_advertiser_id),
            bigquery.ScalarQueryParameter("billing_type", "STRING", billing_type),
        ]
    )

    # Run the query
    query_job = bigquery_client.query(query, job_config=job_config)

    # Wait for the query to complete
    query_job.result()

    print(f"Successfully upserted invoice {invoice_id} with URL {file_url}")

def upload_dataframe_to_bigquery(df, table_id, project_id):
    # client = bigquery.Client(project=project_id)

    # Step 1: Check if the table exists
    try:
        table = bigquery_client.get_table(table_id)
        table_exists = True
        print(f"Table {table_id} exists.")
    except NotFound:
        table_exists = False
        print(f"Table {table_id} does not exist. It will be created.")

    if not table_exists:
        # Define the schema based on the DataFrame's dtypes
        schema = []
        for column in df.columns:
            df[column] = df[column].astype(str)
            field_type = 'STRING'
            schema.append(bigquery.SchemaField(column, field_type))

        # Create the table
        table = bigquery.Table(table_id, schema=schema)
        table = bigquery_client.create_table(table)
        print(f"Created table {table_id} with schema.")

    # Step 2: Delete existing rows with matching invoice IDs
    invoice_ids = df['invoice_id'].dropna().unique().tolist()
    if invoice_ids:
        # Prepare invoice IDs for SQL query
        invoice_ids_str = ', '.join([f"'{str(i)}'" for i in invoice_ids])

        delete_query = f"""
        DELETE FROM `{table_id}`
        WHERE invoice_id IN ({invoice_ids_str})
        """
        query_job = bigquery_client.query(delete_query)
        query_job.result()
        print(f"Deleted existing rows with invoice_id in {invoice_ids}.")
    else:
        print("No invoice_id values found in DataFrame to delete.")

    # Step 3: Insert new data into the table
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND
    )

    load_job = bigquery_client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )
    load_job.result()
    print(f"Inserted {len(df)} rows into {table_id}.")

def generate_encoder_excel(bcm_index):
    sfdc_opp_summary_df = pd.DataFrame(columns=sfdc_opp_summary_columns)
    sfdc_opp_detail_df = pd.DataFrame(columns=sfdc_opp_detail_columns)

    # set key vars
    invoice_target = "encoders"
    invoice_date = datetime.now()
    formatted_invoice_date = invoice_date.strftime("%-m/%-d/%Y")
    summary_group_sort_by = ['invoice_id', 'billing_month', 'encoder_device_location', 'encoder_device_name']
    summary_columns = ['encoder_device_location', 'encoder_device_name', 'li_label', 'li_quantity', 'li_price', 'li_total']
    detail_group_sort_by = ['invoice_id', 'billing_month', 'encoder_device_location', 'encoder_device_name', 'encoding_id']
    detail_columns = ['encoder_device_location', 'encoder_device_name', 'encoding_id', 'isci', 'encoded_for', 'encoded_timestamp']

    date_columns = ['bcw_start_date_display', 'bcw_end_date_display']
    date_time_columns = ['encoded_timestamp']
    currency_columns = ['li_price', 'li_total', 'invoice_total']

    # Define custom column headers
    custom_summary_headers = {
        'encoder_device_location': 'Encoder Location',
        'encoder_device_name': 'Encoder Name',
        'li_label': 'Item',
        'li_quantity': 'Quantity',
        'li_price': 'Rate',
        'li_total': 'Total',
        'invoice_total': 'Invoice Total'
    }

    custom_detail_headers = {
        'encoder_device_location': 'Encoder Location',
        'encoder_device_name': 'Encoder Name',
        'encoding_id': 'Encoding ID',
        'isci': 'ISCI',
        'encoded_for': 'Encoded For',
        'encoded_timestamp': 'Encoded Timestamp UTC'
    }


    # Extract year and month from bcm_index
    year = int(bcm_index)
    # Multiply fractional part by 100 and round to get the month
    month = int(round((bcm_index - year) * 100))

    # Ensure month is between 1 and 12
    if not 1 <= month <= 12:
        raise ValueError("Invalid month extracted from bcm_index.")

    # Convert to 'YYYY-MM' format
    bcm_index_str = f"{year}-{month:02d}"

    # Calculate the first encoding date
    first_encoding_date = f"{bcm_index_str}-01"
    first_month_date = f"{bcm_index_str}-01"
    last_encoding_date = billing_end_date

    # Get the number of days in the month
    num_days = calendar.monthrange(year, month)[1]

    # Calculate the last encoding date
    last_encoding_date = f"{bcm_index_str}-{num_days:02d}"


    # Convert last_encoding_date to a date object
    last_encoding_date_obj = datetime.strptime(last_encoding_date, '%Y-%m-%d').date()

    # Get today's date (using the current date from the initial prompt)

    if today > last_encoding_date_obj:
        print("Today's date is after the last day of the bcm_index month.")


        # Query the summary and detail views
        processed_query = f"""
        SELECT * FROM `{project_id}.{dataset_id}.avs_billing_encodings_prepped`
        """
        # summary_query = """
        # SELECT * FROM `adhoc-billing.avs_billing_process._encoder_billing__line_items_summary`
        # """
        # detail_query = """
        # SELECT * FROM `adhoc-billing.avs_billing_process._encoder_billing__line_items_detail`
        # """

        # Load data into DataFrames
        processed_df = bigquery_client.query(processed_query).to_dataframe()
        processed_df['encoded_timestamp'] = pd.to_datetime(processed_df['encoded_timestamp']).dt.tz_localize(None)
        processed_df = processed_df.sort_values(by=detail_group_sort_by)
        # summary_df = bigquery_client.query(summary_query).to_dataframe()
        # detail_df = bigquery_client.query(detail_query).to_dataframe()


        # summary_df = pd.DataFrame(columns=summary_columns)
        # detail_df = pd.DataFrame(columns=detail_columns)

        # Assuming summary_columns is a list of column names
        mask = processed_df['sfdc_account_id'] == '0013h00001Vc5SCAAZ'
        er_summary_df = processed_df[mask].drop_duplicates(subset=['invoice_id', 'encoder_id']).copy()
        non_er_summary_df = processed_df[~mask].drop_duplicates(subset=['invoice_id', 'li_label']).dropna(subset=['invoice_total']).copy()
        summary_df = pd.concat([er_summary_df, non_er_summary_df])
        summary_df.reset_index(drop=True, inplace=True)

        # summary_df.columns.tolist()
        summary_df[['invoice_id', 'invoice_total_raw', 'min_value']].drop_duplicates()
        summary_df[['sfdc_account_name','encoder_type','encoder_device_name','encoder_device_location']].drop_duplicates()
        summary_df['broadcast_month_name'] = pd.to_datetime(summary_df['billing_month'], format='%Y-%m').dt.strftime('%B-%Y')
        summary_df['op_name'] = summary_df.apply(
            lambda row: f"{row['sfdc_account_name']} - {row['broadcast_month_name']} ({billing_start_date} - {billing_end_date})",
            axis=1
        )
        summary_df['ownerId'] = opp_owner_id
        summary_df['closeDate'] = today.strftime('%Y-%m-%d')
        summary_df['poNumber'] = ''
        summary_df['stageName'] = opp_stage_name
        summary_df['orderdate'] = today.strftime('%Y-%m-%d')
        summary_df['billing_start_date'] = billing_start_date
        summary_df['billing_end_date'] = billing_end_date
        summary_df['type'] = opp_type
        summary_df['BVSInvoiceTerms'] = summary_df['description']
        summary_df['SOSalesRep'] = ''
        summary_df['PriceBook2Id'] = opp_pricebook_id
        summary_df['RecordTypeId'] = opp_record_type_id
        summary_df['invoice_short_url'] = ''
        summary_df['sfdc_opportunity_id'] = ''
        summary_df['sfdc_advertiser_id'] = ''
        summary_df['poNumber'] = ''
        summary_df['reporting_cust_name'] = summary_df['sfdc_account_name']
        summary_df['reporting_adv_name'] = ''
        summary_df['LineDesc'] = summary_df.apply(
            lambda row: f"{row['encoder_type']} : {row['encoder_device_name']} : {row['encoder_device_location']} - {row['li_label']}",
            axis=1
        )
        # summary_df['li_label']
        summary_df['Product2Id'] = summary_df['sfdc_product_id']
        summary_df['ProductCode'] = summary_df['sfdc_product_code']
        summary_df['quantity'] = summary_df['li_quantity']
        summary_df['price'] = summary_df['li_price']

        # Assuming detail_columns is a list of column names

        er_detail_df = processed_df[mask].drop_duplicates(subset=['invoice_id', 'encoder_id', 'encoding_id', 'isci']).copy()
        non_er_detail_df = processed_df[~mask].drop_duplicates(subset=['invoice_id', 'li_label', 'encoding_id', 'isci']).dropna(subset=['invoice_total']).copy()
        detail_df = pd.concat([er_detail_df, non_er_detail_df])
        detail_df.reset_index(drop=True, inplace=True)
        invoice_ids = summary_df['invoice_id'].unique()

        # sfdc_opp_detail_columns = ['invoice_id',  'sfdc_opportunity_id', 'broadcast_month_name', 'LineDesc', 'Product2Id', 'ProductCode', 'quantity', 'price', 'reporting_cust_name', 'reporting_adv_name']

        # Process each invoice_id
        for invoice_id in invoice_ids:
            # Filter summary and detail for the current invoice_id
            summary_group = summary_df[summary_df['invoice_id'] == invoice_id]
            detail_group = detail_df[detail_df['invoice_id'] == invoice_id]

            # Sort summary and detail as needed
            summary_group = summary_group.sort_values(by=summary_group_sort_by)
            if summary_group['min_value'].iloc[0]:
                if summary_group['min_max_applied_at'].iloc[0] == 'account':
                    if summary_group['invoice_total_raw'].iloc[0] < summary_group['min_value'].iloc[0]:
                        summary_group_min_add_df = pd.DataFrame(columns=summary_columns)
                        summary_group_min_add_df = summary_group.iloc[0:1].copy()
                        summary_group_min_add_df['li_label'] = 'Monthly Minimum Adjustment'
                        summary_group_min_add_df['li_quantity'] = 1
                        summary_group_min_add_df['li_price'] = summary_group_min_add_df['min_value'].iloc[0] - summary_group_min_add_df['invoice_total_raw'].iloc[0]
                        summary_group_min_add_df['li_total'] = summary_group_min_add_df['min_value'].iloc[0] - summary_group_min_add_df['invoice_total_raw'].iloc[0]
                        summary_group_min_add_df['LineDesc'] = 'Monthly Minimum Adjustment'
                        summary_group_min_add_df['quantity'] = 1
                        summary_group_min_add_df['price'] = summary_group_min_add_df['min_value'].iloc[0] - summary_group_min_add_df['invoice_total_raw'].iloc[0]
                        summary_group_2 = pd.concat(
                            [summary_group, summary_group_min_add_df],
                            ignore_index=True
                        )
                    else:
                        summary_group_2 = summary_group.copy()
                else:
                    summary_group_2 = summary_group.copy()
            else:
                if summary_group['max_value'].iloc[0]:
                    if summary_group['min_max_applied_at'].iloc[0] == 'account':
                        if summary_group['invoice_total_raw'].iloc[0] > summary_group['max_value'].iloc[0]:
                            summary_group_min_add_df = pd.DataFrame(columns=summary_columns)
                            summary_group_min_add_df = summary_group.iloc[0:1].copy()
                            summary_group_min_add_df['li_label'] = 'Monthly Maximum Adjustment'
                            summary_group_min_add_df['li_quantity'] = 1
                            summary_group_min_add_df['li_price'] = summary_group_min_add_df['max_value'].iloc[0] - summary_group_min_add_df['invoice_total_raw'].iloc[0]
                            summary_group_min_add_df['li_total'] = summary_group_min_add_df['max_value'].iloc[0] - summary_group_min_add_df['invoice_total_raw'].iloc[0]
                            summary_group_min_add_df['LineDesc'] = 'Monthly Maximum Adjustment'
                            summary_group_min_add_df['quantity'] = 1
                            summary_group_min_add_df['price'] = summary_group_min_add_df['max_value'].iloc[0] - summary_group_min_add_df['invoice_total_raw'].iloc[0]
                            summary_group_2 = pd.concat(
                                [summary_group, summary_group_min_add_df],
                                ignore_index=True
                            )
                        else:
                            summary_group_2 = summary_group.copy()
                    else:
                        summary_group_2 = summary_group.copy()
                else:
                    summary_group_2 = summary_group.copy()


            summary_group_2.reset_index(drop=True, inplace=True)
            summary_group_2.columns.to_list()
            # summary_group.drop(columns=['invoice_total_raw'], inplace=True)
            detail_group = detail_group.sort_values(by=detail_group_sort_by)

            # Extract necessary fields for the filename and metadata
            billing_month = summary_group_2['billing_month'].iloc[0]
            currency = summary_group_2['currency'].iloc[0].upper()  # Convert currency to uppercase for consistency
            sfdc_account_name = summary_group_2['sfdc_account_name'].iloc[0]
            sfdc_account_id = summary_group_2['sfdc_account_id'].iloc[0]
            sfdc_advertiser_id = ''
            billing_type = summary_group_2['billing_type'].iloc[0]
            sfdc_account_name_safe = re.sub(r'[^\w\-.]', '_', sfdc_account_name)
            sfdc_account_name_safe = re.sub(r'[|(){}]', '_', sfdc_account_name_safe)
            account_csm = summary_group_2['account_csm'].iloc[0]
            description = summary_group_2['description'].iloc[0]

            # Create an Excel filename
            excel_file_name = f'invoice_{billing_month}_{sfdc_account_name_safe}_{invoice_id}.xlsx'

            # Add encoder details to sfdc opp df
            sfdc_opp_prep_df = pd.DataFrame(columns=sfdc_opp_summary_columns)
            sfdc_opp_prep_df = summary_group_2[sfdc_opp_summary_columns].drop_duplicates(subset=['invoice_id']).copy()
            sfdc_opp_summary_df = pd.concat(
                [sfdc_opp_summary_df, sfdc_opp_prep_df],
                ignore_index=True
            )



            # Create a Pandas Excel writer object
            with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:
                # Define workbook and format objects
                workbook = writer.book
                bold_format = workbook.add_format({'bold': True})
                bold_right_format = workbook.add_format({'bold': True, 'align': 'right'})
                currency_format = workbook.add_format({'num_format': '$#,##0.00'}) if currency == 'USD' else workbook.add_format({'num_format': 'C$#,##0.00'})
                currency_bold_format = workbook.add_format({'bold': True, 'num_format': '$#,##0.00'}) if currency == 'USD' else workbook.add_format({'bold': True, 'num_format': 'C$#,##0.00'})
                date_format = workbook.add_format({'num_format': 'mm/dd/yyyy'})  # Define date format
                date_time_format = workbook.add_format({'num_format': 'mm/dd/yyyy HH:mm:ss'})


                # Write the first 4 lines with invoice_id, sfdc_account_name, invoice date, and billing month
                summary_sheet = workbook.add_worksheet('Summary')
                detail_sheet = workbook.add_worksheet('Details')

                summary_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)
                detail_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)


                headers = [
                    ('Invoice ID:', invoice_id),
                    ('Invoice date:', formatted_invoice_date),
                    ('Account:', sfdc_account_name),
                    ('Billing Month:', billing_month),
                    ('Description:', description)
                ]

                for i, (label, value) in enumerate(headers, start=3):
                    summary_sheet.write(f'A{i}', label, bold_format)
                    summary_sheet.write(f'B{i}', value)
                    detail_sheet.write(f'A{i}', label, bold_format)
                    detail_sheet.write(f'B{i}', value)

                # Write the custom headers in row 8
                for col_num, column in enumerate(summary_columns):
                    custom_header = custom_summary_headers.get(column, column)
                    summary_sheet.write(8, col_num, custom_header, bold_format)

                for col_num, column in enumerate(detail_columns):
                    custom_header = custom_detail_headers.get(column, column)
                    detail_sheet.write(8, col_num, custom_header, bold_format)

                # Start writing data from row 9 (resetting the row number for each new sheet)
                summary_start_row = 9
                detail_start_row = 9

                # Write summary data
                for row_num, row_data in enumerate(summary_group_2.itertuples(index=False), start=summary_start_row):
                    for col_num, column in enumerate(summary_columns):
                        value = getattr(row_data, column)
                        cell_format = (
                            currency_format if column in currency_columns else
                            date_format if column in date_columns else
                            date_time_format if column in date_time_columns else None
                        )
                        summary_sheet.write(row_num, col_num, value, cell_format)
                    sfdc_opp_detail_prep_df = pd.DataFrame(columns=sfdc_opp_detail_columns)
                    sfdc_opp_detail_prep_df = summary_group_2[sfdc_opp_detail_columns].drop_duplicates(subset=['invoice_id','LineDesc']).copy()
                    sfdc_opp_detail_df = pd.concat(
                        [sfdc_opp_detail_df, sfdc_opp_detail_prep_df],
                        ignore_index=True
                    )

                # Write Grand Total
                grand_total_label_row = summary_start_row + len(summary_group_2)
                summary_sheet.write(grand_total_label_row, len(summary_columns) - 4, 'Grand Total:', bold_right_format)
                summary_sheet.write(grand_total_label_row, len(summary_columns) - 3, summary_group_2['invoice_quantity'].iloc[0], bold_right_format)
                summary_sheet.write(grand_total_label_row, len(summary_columns) - 1, summary_group_2['invoice_total'].iloc[0], currency_bold_format)

                # Write detail data
                for row_num, row_data in enumerate(detail_group.itertuples(index=False), start=detail_start_row):
                    for col_num, column in enumerate(detail_columns):
                        value = getattr(row_data, column)
                        cell_format = (
                            currency_format if column in currency_columns else
                            date_format if column in date_columns else
                            date_time_format if column in date_time_columns else None
                        )
                        detail_sheet.write(row_num, col_num, value, cell_format)

                # Auto-fit columns to content starting from A2
                for col_num, column in enumerate(summary_columns):
                    max_length = max(
                        summary_group[column].astype(str).map(len).max(),
                        len(custom_summary_headers.get(column, column))
                    ) + 2
                    summary_sheet.set_column(col_num, col_num, max_length)

                for col_num, column in enumerate(detail_columns):
                    max_length = max(
                        detail_group[column].astype(str).map(len).max(),
                        len(custom_detail_headers.get(column, column))
                    ) + 2
                    detail_sheet.set_column(col_num, col_num, max_length)


            # Upload the file to Google Cloud Storage
            blob_path = f'invoices/{billing_month}/{invoice_target}/{excel_file_name}'
            blob_path2 = f'invoices/{billing_month}/for_csms/{account_csm}/{invoice_target}/{excel_file_name}'
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(excel_file_name)
            blob = bucket.blob(blob_path2)
            blob.upload_from_filename(excel_file_name)

            # Generate signed URL for public access
            file_url = generate_signed_url(bucket_name, blob_path, expiration=30)

            # Upsert the record into BigQuery
            upsert_invoice_record(invoice_id, file_url, bcm_index, sfdc_account_id, sfdc_advertiser_id, billing_type)

            # Optionally, delete the local file after upload
            os.remove(excel_file_name)
    else:
        print("Today's date is before the last day of the bcm_index month.")

    print("Encoding billing Excel files generated, uploaded to GCS, and upserted into BigQuery successfully.")
    sfdc_opp_summary_df['invoice_id'] = sfdc_opp_summary_df['invoice_id'].astype(str)

    upload_dataframe_to_bigquery(sfdc_opp_summary_df, sfdc_opp_summary_table_id, project_id)
    sfdc_opp_detail_df_unique = sfdc_opp_detail_df.drop_duplicates().copy()
    sfdc_opp_detail_df_unique['invoice_id'] = sfdc_opp_detail_df_unique['invoice_id'].astype(str)
    sfdc_opp_detail_df_unique['quantity'] = sfdc_opp_detail_df_unique['quantity'].astype(str)
    sfdc_opp_detail_df_unique['price'] = sfdc_opp_detail_df_unique['price'].astype(str)
    sfdc_opp_detail_df_unique['li_total'] = sfdc_opp_detail_df_unique['li_total'].astype(str)
    upload_dataframe_to_bigquery(sfdc_opp_detail_df_unique, sfdc_opp_detail_table_id, project_id)

def generate_weekly_excel(bcm_index):
    # Create DataFrames
    sfdc_opp_summary_df = pd.DataFrame(columns=sfdc_opp_summary_columns)
    sfdc_opp_detail_df = pd.DataFrame(columns=sfdc_opp_detail_columns)

    # set key vars
    invoice_target = "weekly"
    invoice_date = datetime.now()
    formatted_invoice_date = invoice_date.strftime("%-m/%-d/%Y")
    summary_group_sort_by = ['invoice_id','bcw_index', 'category']
    summary_columns = ['bcw_index', 'bcw_start_date_display', 'bcw_end_date_display', 'category', 'category_quantity', 'rate',  'category_total']
    detail_group_sort_by = ['invoice_id','bcw_index', 'category', 'affiliate', 'callsign']
    detail_exclusion_group_sort_by = ['invoice_id','bcw_index', 'category',  'category_orig', 'affiliate', 'callsign']
    detail_columns = ['bcw_index', 'bcw_start_date_display', 'bcw_end_date_display', 'category', 'affiliate', 'callsign',  'rate', 'encoding_id', 'isci','spot_estimate', 'cable_estimate']
    detail_exclusion_columns = ['bcw_index', 'bcw_start_date_display', 'bcw_end_date_display', 'exclusion_type', 'exclusion_task_id', 'category', 'category_orig', 'affiliate', 'callsign',  'rate', 'encoding_id', 'isci','spot_estimate', 'cable_estimate']


    date_columns = ['bcw_start_date_display', 'bcw_end_date_display']
    currency_columns = ['rate', 'li_total', 'invoice_total', 'category_total']

    weekly_exclusions_df = pd.DataFrame()
    # Define custom column headers
    custom_summary_headers = {
        'bcw_index': 'Broadcast Week Index',
        'bcw_start_date_display': 'Start Date',
        'bcw_end_date_display': 'End Date',
        'category': 'Category',
        'category_quantity': 'Quantity',
        'rate': 'Rate',
        'category_total': 'Total'
    }

    custom_detail_headers = {
        'bcw_index': 'Broadcast Week Index',
        'bcw_start_date_display': 'Start Date',
        'bcw_end_date_display': 'End Date',
        'category': 'Category',
        'affiliate': 'Affiliate',
        'callsign': 'Callsign',
        'rate': 'Rate',
        'encoding_id': 'Encoding ID',
        'isci': 'ISCI',
        'spot_estimate': 'Spot Estimate',
        'cable_estimate': 'Cable Estimate'
    }

    custom_detail_headers_exclusions = {
        'bcw_index': 'Broadcast Week Index',
        'bcw_start_date_display': 'Start Date',
        'bcw_end_date_display': 'End Date',
        'exclusion_type': 'Exclusion Type',
        'exclusion_task_id': 'Exclusion Task ID',
        'category': 'Category',
        'category_orig': 'Original Category',
        'affiliate': 'Affiliate',
        'callsign': 'Callsign',
        'rate': 'Rate',
        'encoding_id': 'Encoding ID',
        'isci': 'ISCI',
        'spot_estimate': 'Spot Estimate',
        'cable_estimate': 'Cable Estimate'
    }


    # Query the summary and detail views
    processed_query = f"""
    SELECT * FROM `{project_id}.{dataset_id}._weekly_processed_inclusions`
    """

    # summary_query = """
    # SELECT * FROM `adhoc-billing.avs_billing_process._weekly_billing_invoice_items_summary`
    # """
    # detail_query = """
    # SELECT * FROM `adhoc-billing.avs_billing_process._weekly_billing_invoice_items_detail`
    # """

    # Load data into DataFrames
    processed_df = bigquery_client.query(processed_query).to_dataframe()
    processed_df = processed_df.sort_values(by=detail_group_sort_by)
    processed_df['excluded'] = processed_df['excluded'].astype(bool)
    excluded_mask = processed_df['excluded'] == True
    weekly_exclusions_df = processed_df[excluded_mask].copy()
    weekly_exclusions_df['exclusion_type'] = weekly_exclusions_df['exclusion_details'].apply(lambda x: x['exclusion_type'])
    weekly_exclusions_df['exclusion_task_id'] = weekly_exclusions_df['exclusion_details'].apply(lambda x: x['exclusion_task_id'])
    weekly_exclusions_df.reset_index(drop=True, inplace=True)
    weekly_include_df = processed_df[~excluded_mask].copy()
    weekly_include_df.reset_index(drop=True, inplace=True)
    # if len(weekly_exclusions_df) > 0 :
    #     has_exclusions = True
    # else:
    #     has_exclusions = False
    # weekly_exclude_df.reset_index(drop=True, inplace=True)

    # weekly_include_df.reset_index(drop=True, inplace=True)





    # processed_df.dtypes
    # array_columns = [col for col in processed_df.columns if isinstance(processed_df[col].iloc[0], np.ndarray)]
    # print("Columns containing numpy arrays:", array_columns)
    # for col in ['encodings_array', 'isci_array', 'invoice_iscis', 'invoice_encodings']:
    #     # Apply a function to each column: convert the numpy array to a list of unique values
    #     processed_df[col] = processed_df[col].apply(lambda x: list(np.unique(x)))


    # list_columns = [col for col in processed_df.columns if isinstance(processed_df[col].iloc[0], list)]
    # print("Columns containing lists:", list_columns)
    # processed_df['']
    # summary_df_orig = bigquery_client.query(summary_query).to_dataframe()
    # summary_df.columns.to_list()
    # detail_df = bigquery_client.query(detail_query).to_dataframe()
    # detail_df.columns.to_list()
    # Get unique invoice_ids
    invoice_ids = weekly_include_df['invoice_id'].unique()
    # summary_df = pd.DataFrame(columns=summary_columns)
    # detail_df = pd.DataFrame(columns=detail_columns)
    # summary_df = pd.DataFrame(columns=summary_columns)

    # Assuming summary_columns is a list of column names
    summary_df = weekly_include_df.drop_duplicates(subset=['invoice_id', 'bcw_index', 'category']).copy()
    summary_df.reset_index(drop=True, inplace=True)
    # summary_df.columns.tolist()
    # summary_df['billing_month']
    summary_df['broadcast_month_name'] = pd.to_datetime(summary_df['billing_month'], format='%Y-%m').dt.strftime('%B-%Y')
    summary_df['op_name'] = summary_df.apply(
        lambda row: f"{row['sfdc_account_name']} - {row['invoice_title']} ({row['bcm_start_date_display']} - {row['bcm_end_date_display']})",
        axis=1
    )
    summary_df['ownerId'] = opp_owner_id
    summary_df['closeDate'] = today.strftime('%Y-%m-%d')
    summary_df['poNumber'] = ''
    summary_df['stageName'] = opp_stage_name
    summary_df['orderdate'] = today.strftime('%Y-%m-%d')
    summary_df['billing_start_date'] = billing_start_date
    summary_df['billing_end_date'] = billing_end_date
    summary_df['type'] = opp_type
    summary_df['BVSInvoiceTerms'] = summary_df['description']
    summary_df['SOSalesRep'] = ''
    summary_df['PriceBook2Id'] = opp_pricebook_id
    summary_df['RecordTypeId'] = opp_record_type_id
    summary_df['invoice_short_url'] = ''
    summary_df['sfdc_opportunity_id'] = ''
    # summary_df['sfdc_advertiser_id'] =
    summary_df['poNumber'] = ''
    summary_df['reporting_cust_name'] = summary_df['sfdc_account_name']
    summary_df['reporting_adv_name'] = summary_df['sfdc_advertiser_id']
    summary_df['LineDesc'] = summary_df.apply(
        lambda row: f"{row['bcw_index']} : {row['bcw_start_date_display']} - {row['bcw_end_date_display']} : {row['category']}",
        axis=1
    )
    # summary_df['li_label']
    summary_df['Product2Id'] = summary_df['sfdc_product_id']
    summary_df['ProductCode'] = summary_df['sfdc_product_code']
    summary_df['quantity'] = summary_df['category_quantity']
    summary_df['price'] = summary_df['rate']

    # summary_df[['category','quantity', 'rate', 'category_quantity', 'category_total']]

    detail_df = weekly_include_df.drop_duplicates(subset=['invoice_id', 'bcw_index', 'category', 'affiliate', 'callsign', 'isci']).copy()
    detail_df.reset_index(drop=True, inplace=True)

    if len(weekly_exclusions_df) > 0:
        detail_exclusions_df = weekly_exclusions_df.drop_duplicates(subset=['invoice_id', 'bcw_index', 'category', 'category_orig', 'affiliate', 'callsign', 'isci', 'spot_estimate', 'cable_estimate']).copy()
        detail_exclusions_df.reset_index(drop=True, inplace=True)
        has_weekly_exclusions = True
    else:
        detail_exclusions_df = pd.DataFrame()
        has_weekly_exclusions = False


    # Process each invoice_id
    for invoice_id in invoice_ids:
        # Filter summary and detail for the current invoice_id
        summary_group = summary_df[summary_df['invoice_id'] == invoice_id]
        detail_group = detail_df[detail_df['invoice_id'] == invoice_id]
        # invoice_index = summary_group['invoice_index'].iloc[0]
        if has_weekly_exclusions:
            detail_exclusions_group = (detail_exclusions_df[detail_exclusions_df['invoice_id'] == invoice_id]).copy()
            # print((detail_exclusions_group))

            if len(detail_exclusions_group) > 0:
                detail_exclusions_group.reset_index(drop=True, inplace=True)
                detail_exclusions_group = detail_exclusions_group.sort_values(by=detail_exclusion_group_sort_by)
                detail_exclusions_group['excluded'] = detail_exclusions_group['excluded'].astype(bool)
                has_invoice_exclusions = True
            else:
                has_invoice_exclusions = False
        else:
            has_invoice_exclusions = False

        # Sort summary and detail as needed
        summary_group = summary_group.sort_values(by=summary_group_sort_by)
        detail_group = detail_group.sort_values(by=detail_group_sort_by)

        # Extract necessary fields for the filename
        bcm_index = summary_group['bcm_index'].iloc[0]
        bcm_name = summary_group['bcm_name'].iloc[0]
        bcm_start = summary_group['bcm_start_date_display'].iloc[0]
        bcm_end = summary_group['bcm_end_date_display'].iloc[0]
        bcm_dates = f'{bcm_start} to {bcm_end}'
        billing_month = summary_group['billing_month'].iloc[0]
        currency = summary_group['currency'].iloc[0].upper()  # Convert currency to uppercase for consistency
        sfdc_account_name = summary_group['sfdc_account_name'].iloc[0]
        sfdc_account_id = summary_group['sfdc_account_id'].iloc[0]
        sfdc_advertiser_id = ''
        billing_type = 'weekly'
        sfdc_account_name_safe = re.sub(r'[^\w\-.]', '_', sfdc_account_name)
        sfdc_account_name_safe = re.sub(r'[|(){}]', '_', sfdc_account_name_safe)
        invoice_title = summary_group['invoice_title'].iloc[0]
        invoice_title_safe = re.sub(r'[^\w\-.]', '_', invoice_title)
        invoice_title_safe = re.sub(r'[|(){}]', '_', invoice_title_safe)
        account_csm = summary_group['csm_sfdc_user_name'].iloc[0]
        description = summary_group['description'].iloc[0]

        # Create an Excel filename
        excel_file_name = f'invoice_{billing_month}_{sfdc_account_name_safe}_{invoice_title_safe}_{invoice_id}.xlsx'

        # Add weekly details to sfdc opp df
        sfdc_opp_prep_df = pd.DataFrame(columns=sfdc_opp_summary_columns)
        sfdc_opp_prep_df = summary_group[sfdc_opp_summary_columns].drop_duplicates(subset=['invoice_id']).copy()
        sfdc_opp_summary_df = pd.concat(
            [sfdc_opp_summary_df, sfdc_opp_prep_df],
            ignore_index=True
        )

        # Create a Pandas Excel writer object
        with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:
            # Define workbook and format objects
            workbook = writer.book
            bold_format = workbook.add_format({'bold': True})
            bold_right_format = workbook.add_format({'bold': True, 'align': 'right'})
            currency_format = workbook.add_format({'num_format': '$#,##0.00'}) if currency == 'USD' else workbook.add_format({'num_format': 'C$#,##0.00'})
            currency_bold_format = workbook.add_format({'bold': True, 'num_format': '$#,##0.00'}) if currency == 'USD' else workbook.add_format({'bold': True, 'num_format': 'C$#,##0.00'})
            date_format = workbook.add_format({'num_format': 'mm/dd/yyyy'})  # Define date format

            # Write the company name at the top
            summary_sheet = workbook.add_worksheet('Summary')
            detail_sheet = workbook.add_worksheet('Details')


            summary_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)
            detail_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)

            if has_invoice_exclusions:
                detail_exclusions_sheet = workbook.add_worksheet('Exclusions')
                detail_exclusions_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)

            # Write the first 4 lines with invoice_id, sfdc_account_name, invoice date, and billing month
            headers = [
                ('Invoice ID:', invoice_id),
                ('Invoice Date:', formatted_invoice_date),
                ('Account:', sfdc_account_name),
                ('Broadcast Month:', bcm_name),
                ('Broadcast Month Dates:', bcm_dates),
                (invoice_title, ''),
                ('Description:', description)
            ]

            for i, (label, value) in enumerate(headers, start=3):  # Start from row 3 to leave space for the company name
                summary_sheet.write(f'A{i}', label, bold_format)
                summary_sheet.write(f'B{i}', value)
                detail_sheet.write(f'A{i}', label, bold_format)
                detail_sheet.write(f'B{i}', value)
                if has_invoice_exclusions:
                    detail_exclusions_sheet.write(f'A{i}', label, bold_format)
                    detail_exclusions_sheet.write(f'B{i}', value)

            # Write the custom headers in row 10
            for col_num, column in enumerate(summary_columns):
                custom_header = custom_summary_headers.get(column, column)
                summary_sheet.write(10, col_num, custom_header, bold_format)


            for col_num, column in enumerate(detail_columns):
                custom_header = custom_detail_headers.get(column, column)
                detail_sheet.write(10, col_num, custom_header, bold_format)

            if has_invoice_exclusions:
                for col_num, column in enumerate(detail_exclusion_columns):
                    custom_header = custom_detail_headers_exclusions.get(column, column)
                    detail_exclusions_sheet.write(10, col_num, custom_header, bold_format)

            # Start writing data from row 11
            summary_start_row = 11
            detail_start_row = 11
            category_grand_quantity = 0
            category_grand_total = 0
            # Write summary data
            for row_num, row_data in enumerate(summary_group.itertuples(index=False), start=summary_start_row):

                for col_num, column in enumerate(summary_columns):
                    value = getattr(row_data, column)
                    cell_format = (
                        currency_format if column in currency_columns else
                        date_format if column in date_columns else None
                    )
                    summary_sheet.write(row_num, col_num, value, cell_format)
                    if column == 'category_quantity':
                        category_grand_quantity = category_grand_quantity + value
                    if column == 'category_total':
                        category_grand_total = category_grand_total + value
                sfdc_opp_detail_prep_df = pd.DataFrame(columns=sfdc_opp_detail_columns)
                sfdc_opp_detail_prep_df = summary_group[sfdc_opp_detail_columns].drop_duplicates(subset=['invoice_id','LineDesc']).copy()
                sfdc_opp_detail_df = pd.concat(
                    [sfdc_opp_detail_df, sfdc_opp_detail_prep_df],
                    ignore_index=True
                )

            # Write Grand Total
            grand_total_label_row = summary_start_row + len(summary_group)
            summary_sheet.write(grand_total_label_row, len(summary_columns) - 4, 'Grand Total:', bold_right_format)
            summary_sheet.write(grand_total_label_row, len(summary_columns) - 3, category_grand_quantity, bold_right_format)
            summary_sheet.write(grand_total_label_row, len(summary_columns) - 1, category_grand_total, currency_bold_format)

            # Write detail data
            for row_num, row_data in enumerate(detail_group.itertuples(index=False), start=detail_start_row):
                for col_num, column in enumerate(detail_columns):
                    value = getattr(row_data, column)
                    cell_format = (
                        currency_format if column in currency_columns else
                        date_format if column in date_columns else None
                    )
                    detail_sheet.write(row_num, col_num, value, cell_format)

            if has_invoice_exclusions:
                # Write detail data
                for row_num, row_data in enumerate(detail_exclusions_group.itertuples(index=False), start=detail_start_row):
                    for col_num, column in enumerate(detail_exclusion_columns):
                        value = getattr(row_data, column)
                        cell_format = (
                            currency_format if column in currency_columns else
                            date_format if column in date_columns else None
                        )
                        detail_exclusions_sheet.write(row_num, col_num, value, cell_format)

            # Auto-fit columns to content starting from A2
            for col_num, column in enumerate(summary_columns):
                max_length = max(
                    summary_group[column].astype(str).map(len).max(),
                    len(custom_summary_headers.get(column, column))
                ) + 6
                summary_sheet.set_column(col_num, col_num, max_length)

            for col_num, column in enumerate(detail_columns):
                max_length = max(
                    detail_group[column].astype(str).map(len).max(),
                    len(custom_detail_headers.get(column, column))
                ) + 6
                detail_sheet.set_column(col_num, col_num, max_length)

            if has_invoice_exclusions:
                for col_num, column in enumerate(detail_exclusion_columns):
                    max_length = max(
                    detail_exclusions_group[column].astype(str).map(len).max(),
                    len(custom_detail_headers.get(column, column))
                    ) + 6
                    detail_exclusions_sheet.set_column(col_num, col_num, max_length)

        # Upload the file to Google Cloud Storage
        blob_path = f'invoices/{billing_month}/{invoice_target}/{excel_file_name}'
        blob = bucket.blob(blob_path)
        blob.upload_from_filename(excel_file_name)
        blob2_path = f'invoices/{billing_month}/for_csms/{account_csm}/{invoice_target}/{excel_file_name}'
        blob = bucket.blob(blob2_path)
        blob.upload_from_filename(excel_file_name)

        # Generate signed URL for public access
        file_url = generate_signed_url(bucket_name, blob_path, expiration=30)

        # Upsert the record into BigQuery
        upsert_invoice_record(invoice_id, file_url, bcm_index, sfdc_account_id, sfdc_advertiser_id, billing_type)

        # Optionally, delete the local file after upload
        os.remove(excel_file_name)

    print("Weekly billing Excel files generated, uploaded to GCS, and upserted into BigQuery successfully.")


    sfdc_opp_summary_df['invoice_id'] = sfdc_opp_summary_df['invoice_id'].astype(str)
    upload_dataframe_to_bigquery(sfdc_opp_summary_df, sfdc_opp_summary_table_id, project_id)

    sfdc_opp_detail_df_unique = sfdc_opp_detail_df.drop_duplicates().copy()
    sfdc_opp_detail_df_unique['invoice_id'] = sfdc_opp_detail_df_unique['invoice_id'].astype(str)
    sfdc_opp_detail_df_unique['quantity'] = sfdc_opp_detail_df_unique['quantity'].astype(str)
    sfdc_opp_detail_df_unique['price'] = sfdc_opp_detail_df_unique['price'].astype(str)
    sfdc_opp_detail_df_unique['li_total'] = sfdc_opp_detail_df_unique['li_total'].astype(str)
    upload_dataframe_to_bigquery(sfdc_opp_detail_df_unique, sfdc_opp_detail_table_id, project_id)

def generate_monthly_excel(bcm_index):
    # set key vars
    invoice_target = "monthly"

    sfdc_opp_summary_df = pd.DataFrame(columns=sfdc_opp_summary_columns)
    sfdc_opp_detail_df = pd.DataFrame(columns=sfdc_opp_detail_columns)


    invoice_date = datetime.now()
    formatted_invoice_date = invoice_date.strftime("%-m/%-d/%Y")
    summary_group_sort_by = ['bcm_index', 'sfdc_account_name', 'sfdc_advertiser_name']
    summary_columns = ['li_label', 'li_quantity', 'li_price', 'li_total']
    advertiser_summary_columns = ['sfdc_advertiser_name','li_label', 'li_quantity', 'li_price', 'li_total']
    detail_group_sort_by = ['bcm_index', 'sfdc_account_name', 'sfdc_advertiser_name', 'isci']
    detail_exclusion_group_sort_by = ['invoice_id','bcm_index', 'sfdc_account_name', 'sfdc_advertiser_name', 'isci']
    detail_columns = ['sfdc_advertiser_name', 'encoding_id', 'isci']
    detail_exclusion_columns = [ 'sfdc_advertiser_name', 'encoding_id', 'isci', 'exclusion_type', 'exclusion_task_id']


    date_columns = ['bcm_start_date_display', 'bcm_end_date_display']
    currency_columns = ['li_price', 'li_total', 'inv_total']
    quantity_columns = ['li_qty', 'adv_iscis', 'inv_iscis', 'isci_detections', 'inv_detections']

    monthly_exclusions_df = pd.DataFrame()
    # Define custom column headers
    custom_summary_headers = {
        'li_label': 'Item',
        'li_qty': 'Quantity',
        'li_price': 'Price',
        'li_total': 'Total',
    }

    advertiser_custom_summary_headers = {
        'sfdc_advertiser_name': '',
        'li_label': 'Item',
        'li_qty': 'Quantity',
        'li_price': 'Price',
        'li_total': 'Total',
    }

    custom_detail_headers = {

        'sfdc_advertiser_name': 'Advertiser',
        'encoding_id': 'Encoding ID',
        'isci': 'ISCI',

    }

    custom_detail_headers_exclusions = {
        'sfdc_advertiser_name': 'Advertiser Name',
        'encoding_id': 'Encoding ID',
        'isci': 'ISCI',
        'exclusion_type': 'Exclusion Type',
        'exclusion_task_id': 'Exclusion Task ID',

    }



        # Query the summary and detail views

    monthly_processed_query = f"""
    SELECT * FROM `{project_id}.{dataset_id}._monthly_processed_inclusions`
    """
    summary_query = f"""
    SELECT * FROM `{project_id}.{dataset_id}._monthly_billing_invoice_items_usage___summary` where bcm_index = {bcm_index}
    """
    # detail_query = f"""
    # SELECT * FROM `adhoc-billing.avs_billing_process._monthly_billing_invoice_items_usage___detail_archive` where bcm_index = {bcm_index}
    # """
    detail_isci_counts_query = f"""
    SELECT * FROM `{project_id}.{dataset_id}._monthly_billing_invoice_items_usage___detail_isci_counts`
    """
    detail_detection_counts_query = f"""
    SELECT * FROM `{project_id}.{dataset_id}._monthly_billing_invoice_items_usage___detail_detection_counts`
    """
    # # Display all columns
    # pd.set_option('display.max_columns', None)

    # # Set unlimited column width
    # pd.set_option('display.max_colwidth', None)
    # Load data into DataFrames
    monthly_processed_df = bigquery_client.query(monthly_processed_query).to_dataframe()

    monthly_processed_df[monthly_processed_df['format_id'] == 15265]
    monthly_processed_df = monthly_processed_df.sort_values(by=detail_group_sort_by)
    monthly_processed_df['excluded'] = monthly_processed_df['excluded'].astype(bool)
    excluded_mask = monthly_processed_df['excluded'] == True
    monthly_exclusions_df = monthly_processed_df[excluded_mask].copy()
    monthly_exclusions_df['exclusion_type'] = monthly_exclusions_df['exclusion_details'].apply(lambda x: x['exclusion_type'])
    monthly_exclusions_df['exclusion_task_id'] = monthly_exclusions_df['exclusion_details'].apply(lambda x: x['exclusion_task_id'])
    monthly_exclusions_df.reset_index(drop=True, inplace=True)
    monthly_include_df = monthly_processed_df[~excluded_mask].copy()
    monthly_include_df.reset_index(drop=True, inplace=True)
    monthly_include_df[monthly_include_df['format_id'] == 15265]

    # monthly_include_df['li_label'] = monthly_include_df.apply(
    #     lambda row: f"{row['sfdc_advertiser_name']} : {row['li_label']}" if row['sfdc_account_id'] == '0013h00001Vc5S1AAJ' else f"{row['li_label']}",
    #     axis=1
    # )

    monthly_exclusions_df['sfdc_account_name'].drop_duplicates()

    # summary_df = bigquery_client.query(summary_query).to_dataframe()
    # detail_df = bigquery_client.query(detail_query).to_dataframe()
    detail_isci_counts_df = bigquery_client.query(detail_isci_counts_query).to_dataframe()
    detail_isci_counts_df[detail_isci_counts_df['format_id'] == 15265]
    detail_detection_counts_df = bigquery_client.query(detail_detection_counts_query).to_dataframe()
    detail_detection_counts_df[detail_detection_counts_df['format_id'] == 15265]

    # Get unique invoice_ids

    monthly_include_df['currency'].fillna('USD', inplace=True)
    # advertiser_breakout_mask = monthly_include_df['invoice_format'] == 'account-advertiser_invoice'
    # standard_monthly_include_df = monthly_include_df[~advertiser_breakout_mask].copy()
    # standard_monthly_include_df['currency'].fillna('USD', inplace=True)
    # standard_monthly_include_df.reset_index(drop=True, inplace=True)
    # advertiser_monthly_include_df = monthly_include_df[advertiser_breakout_mask].copy()
    # advertiser_monthly_include_df['currency'].fillna('USD', inplace=True)
    # advertiser_monthly_include_df.reset_index(drop=True, inplace=True)

    # Assuming summary_columns is a list of column names
    summary_df = monthly_include_df.drop_duplicates(subset=['invoice_id', 'li_label', 'li_total', 'li_quantity']).dropna(subset=['li_quantity']).copy()
    summary_df[summary_df['format_id'] == 15265]
    summary_df.reset_index(drop=True, inplace=True)
    invoice_ids = summary_df['invoice_id'].unique()
    # advertiser_summary_df = monthly_include_df.drop_duplicates(subset=['invoice_id', 'sfdc_advertiser_id', 'li_label', 'li_total', 'li_quantity']).copy()
    # advertiser_summary_df.reset_index(drop=True, inplace=True)

    summary_df.columns.to_list()
    summary_df['invoice_format'].drop_duplicates()
    summary_df['usage_tier_plat_detection_billing_unit'].drop_duplicates()

    summary_df.columns.tolist()
    summary_df['li_total']
    summary_df['broadcast_month_name'] = pd.to_datetime(summary_df['billing_month'], format='%Y-%m').dt.strftime('%B-%Y')
    summary_df['op_name'] = summary_df.apply(
        lambda row: f"{row['sfdc_account_name']} - {row['invoice_title']} ({row['bcm_start_date_display']} - {row['bcm_end_date_display']})",
        axis=1
    )
    summary_df.columns.to_list()
    summary_df['ownerId'] = opp_owner_id
    summary_df['closeDate'] = today.strftime('%Y-%m-%d')
    summary_df['poNumber'] = ''
    summary_df['stageName'] = opp_stage_name
    summary_df['orderdate'] = today.strftime('%Y-%m-%d')
    summary_df['billing_start_date'] = billing_start_date
    summary_df['billing_end_date'] = billing_end_date
    summary_df['type'] = opp_type
    summary_df['BVSInvoiceTerms'] = summary_df['description']
    summary_df['SOSalesRep'] = ''
    summary_df['PriceBook2Id'] = opp_pricebook_id
    summary_df['RecordTypeId'] = opp_record_type_id
    summary_df['invoice_short_url'] = ''
    summary_df['sfdc_opportunity_id'] = ''
    # summary_df['sfdc_advertiser_id'] =
    # summary_df['poNumber'] = ''
    summary_df['reporting_cust_name'] = summary_df['sfdc_account_name']
    summary_df['reporting_adv_name'] = summary_df['sfdc_advertiser_name']

    summary_df['LineDesc'] = summary_df['li_label']

    # summary_df['li_label']
    summary_df['Product2Id'] = summary_df['sfdc_product_id']
    summary_df['ProductCode'] = summary_df['sfdc_product_code']
    summary_df['quantity'] = summary_df['li_quantity']
    summary_df['price'] = summary_df['li_price']


    detail_df = monthly_include_df.drop_duplicates(subset=['invoice_id', 'bcm_index', 'sfdc_account_id', 'sfdc_advertiser_id', 'isci']).dropna(subset=['li_quantity']).copy()
    detail_df.reset_index(drop=True, inplace=True)
    detail_df.columns.to_list()
    # detail_df[['invoice_id', 'bcm_index', 'sfdc_advertiser_id', 'isci','detections']].drop_duplicates()

    # exclusions_df = monthly_exclusions_df.drop_duplicates(subset=['invoice_id', 'bcm_index', 'sfdc_account_id', 'sfdc_advertiser_id', 'isci']).copy()

    if len(monthly_exclusions_df) > 0:
        detail_exclusions_df = monthly_exclusions_df.drop_duplicates(subset=['invoice_id', 'bcm_index', 'sfdc_account_id', 'sfdc_advertiser_id', 'isci']).copy()
        detail_exclusions_df.reset_index(drop=True, inplace=True)
        has_monthly_exclusions = True
    else:
        detail_exclusions_df = pd.DataFrame()
        has_monthly_exclusions = False

    # Extract necessary fields for the filename
    bcm_index = summary_df['bcm_index'].iloc[0]
    bcm_name = summary_df['bcm_name'].iloc[0]
    bcm_start = summary_df['bcm_start_date_display'].iloc[0]
    bcm_end = summary_df['bcm_end_date_display'].iloc[0]
    bcm_dates = f'{bcm_start} to {bcm_end}'
    billing_month = summary_df['billing_month'].iloc[0]
    summary_group = pd.DataFrame()
    advertiser_summary_group = pd.DataFrame()

    # Process each invoice_id
    for invoice_id in invoice_ids:
        # Filter summary and detail for the current invoice_id
        summary_group = summary_df[summary_df['invoice_id'] == invoice_id]
        if summary_group['min_value'].iloc[0]:
            if summary_group['min_applied_at'].iloc[0] == 'account':
                if summary_group['inv_total_raw'].iloc[0] < summary_group['min_value'].iloc[0]:
                    summary_group_min_add_df = pd.DataFrame(columns=summary_columns)
                    summary_group_min_add_df = summary_group.iloc[0:1].copy()
                    summary_group_min_add_df['li_label'] = 'Monthly Minimum Adjustment'
                    summary_group_min_add_df['li_quantity'] = 1
                    summary_group_min_add_df['li_price'] = summary_group_min_add_df['min_value'].iloc[0] - summary_group_min_add_df['inv_total_raw'].iloc[0]
                    summary_group_min_add_df['li_total'] = summary_group_min_add_df['min_value'].iloc[0] - summary_group_min_add_df['inv_total_raw'].iloc[0]
                    summary_group_min_add_df['LineDesc'] = 'Monthly Minimum Adjustment'
                    summary_group_min_add_df['quantity'] = 1
                    summary_group_min_add_df['price'] = summary_group_min_add_df['min_value'].iloc[0] - summary_group_min_add_df['inv_total_raw'].iloc[0]
                    summary_group_2 = pd.concat(
                        [summary_group, summary_group_min_add_df],
                        ignore_index=True
                    )
                else:
                    summary_group_2 = summary_group.copy()
            else:
                summary_group_2 = summary_group.copy()
        else:
            if summary_group['max_value'].iloc[0]:
                if summary_group['max_applied_at'].iloc[0] == 'account':
                    if summary_group['inv_total_raw'].iloc[0] > summary_group['max_value'].iloc[0]:
                        summary_group_min_add_df = pd.DataFrame(columns=summary_columns)
                        summary_group_min_add_df = summary_group.iloc[0:1].copy()
                        summary_group_min_add_df['li_label'] = 'Monthly Maximum Adjustment'
                        summary_group_min_add_df['li_quantity'] = 1
                        summary_group_min_add_df['li_price'] = summary_group_min_add_df['max_value'].iloc[0] - summary_group_min_add_df['inv_total_raw'].iloc[0]
                        summary_group_min_add_df['li_total'] = summary_group_min_add_df['max_value'].iloc[0] - summary_group_min_add_df['inv_total_raw'].iloc[0]
                        summary_group_min_add_df['LineDesc'] = 'Monthly Maximum Adjustment'
                        summary_group_min_add_df['quantity'] = 1
                        summary_group_min_add_df['price'] = summary_group_min_add_df['max_value'].iloc[0] - summary_group_min_add_df['inv_total_raw'].iloc[0]
                        summary_group_2 = pd.concat(
                            [summary_group, summary_group_min_add_df],
                            ignore_index=True
                        )
                    else:
                        summary_group_2 = summary_group.copy()
                else:
                    summary_group_2 = summary_group.copy()
            else:
                summary_group_2 = summary_group.copy()


        summary_group_2.reset_index(drop=True, inplace=True)
        summary_group = summary_group_2.copy()
        # advertiser_summary_group = advertiser_summary_df[advertiser_summary_df['invoice_id'] == invoice_id]
        detail_group = detail_df[detail_df['invoice_id'] == invoice_id]
        exclusions_group = monthly_exclusions_df[monthly_exclusions_df['invoice_id'] == invoice_id]

        new_invoice_total = 0

        if len(exclusions_group) > 0:
            has_monthly_invoice_exclusions = True
        else:
            has_monthly_invoice_exclusions = False

        detail_isci_counts_group = detail_isci_counts_df[detail_isci_counts_df['invoice_id'] == invoice_id]

        detail_detection_counts_group = detail_detection_counts_df[detail_detection_counts_df['invoice_id'] == invoice_id]

        # Extract 'inv_iscis' numeric value for the current invoice_id
        try:
            inv_iscis = detail_isci_counts_group['inv_iscis'].iloc[0]
            advertiser_inv_iscis = advertiser_summary_group[['adv_iscis']].sum()
        except:
            inv_iscis = 0  # Set to 0 if 'inv_iscis' is not found

        include_detections = False
        # Extract 'inv_detections' numeric value for the current invoice_id

        try:
            inv_detections = detail_detection_counts_df['inv_detections'].iloc[0]
            # inv_detections = summary_group['isci_detections'].sum()
            include_detections = True
        except:
            inv_detections = 0  # Set to 0 if 'inv_detections' is not found
            include_detections = False

        total_iscis_inserted = False
        total_detections_inserted = False

        if has_monthly_exclusions:
            detail_exclusions_group = (detail_exclusions_df[detail_exclusions_df['invoice_id'] == invoice_id]).copy()
            # print((detail_exclusions_group))

            if len(detail_exclusions_group) > 0:
                detail_exclusions_group.reset_index(drop=True, inplace=True)
                detail_exclusions_group = detail_exclusions_group.sort_values(by=detail_exclusion_group_sort_by)
                detail_exclusions_group['excluded'] = detail_exclusions_group['excluded'].astype(bool)
                has_invoice_exclusions = True
            else:
                has_invoice_exclusions = False
        else:
            has_invoice_exclusions = False

        # Sort summary and detail as needed
        summary_group = summary_group.sort_values(by=summary_group_sort_by)
        detail_group = detail_group.sort_values(by=detail_group_sort_by)
        currency = summary_group['currency'].iloc[0].upper()  # Convert currency to uppercase for consistency
        sfdc_account_name = summary_group['sfdc_account_name'].iloc[0]
        sfdc_account_id = summary_group['sfdc_account_id'].iloc[0]
        sfdc_advertiser_id = summary_group['sfdc_advertiser_id'].iloc[0]
        billing_type = summary_group['billing_type'].iloc[0]
        sfdc_account_name_safe = re.sub(r'[^\w\-.]', '_', sfdc_account_name)
        sfdc_account_name_safe = re.sub(r'[|(){}]', '_', sfdc_account_name_safe)
        invoice_title = summary_group['invoice_title'].iloc[0]
        invoice_title_safe = re.sub(r'[^\w\-.]', '_', invoice_title)
        invoice_title_safe = re.sub(r'[|(){}]', '_', invoice_title_safe)
        description = summary_group['description'].iloc[0]
        account_csm = summary_group['csm_sfdc_user_name'].iloc[0]


        # Create an Excel filename
        excel_file_name = f'invoice_{billing_month}_{sfdc_account_name_safe}_{invoice_title_safe}_{invoice_id}.xlsx'

        # Add weekly details to sfdc opp df
        sfdc_opp_prep_df = pd.DataFrame(columns=sfdc_opp_summary_columns)
        sfdc_opp_prep_df = summary_group[sfdc_opp_summary_columns].drop_duplicates(subset=['invoice_id']).copy()
        sfdc_opp_summary_df = pd.concat(
            [sfdc_opp_summary_df, sfdc_opp_prep_df],
            ignore_index=True
        )

        # Create a Pandas Excel writer object
        with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:
            # Define workbook and format objects
            workbook = writer.book
            bold_format = workbook.add_format({'bold': True})
            bold_right_format = workbook.add_format({'bold': True, 'align': 'right'})
            quantity_format = workbook.add_format({'num_format': '#,##0'})
            quantity_bold_format = workbook.add_format({'bold': True, 'num_format': '#,##0'})
            currency_format = workbook.add_format({'num_format': '$#,##0.00'}) if currency == 'USD' else workbook.add_format({'num_format': 'C$#,##0.00'})
            currency_bold_format = workbook.add_format({'bold': True, 'num_format': '$#,##0.00'}) if currency == 'USD' else workbook.add_format({'bold': True, 'num_format': 'C$#,##0.00'})
            date_format = workbook.add_format({'num_format': 'mm/dd/yyyy'})  # Define date format

            # Write the company name at the top
            summary_sheet = workbook.add_worksheet('Summary')
            detail_sheet = workbook.add_worksheet('Details')

            summary_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)
            detail_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)

            if has_invoice_exclusions:
                detail_exclusions_sheet = workbook.add_worksheet('Exclusions')
                detail_exclusions_sheet.write('A1', "Veil Global Technologies (formerly Advocado)", bold_format)


            # Write the first 4 lines with invoice_id, sfdc_account_name, invoice date, and billing month
            headers = [
                ('Invoice ID:', invoice_id),
                ('Invoice Date:', formatted_invoice_date),
                ('Account:', sfdc_account_name),
                ('Broadcast Month:', bcm_name),
                ('Broadcast Month Dates:', bcm_dates),
                (invoice_title, ''),
                ('Description:', description)
            ]
            for i, (label, value) in enumerate(headers, start=3):  # Start from row 3 to leave space for the company name
                summary_sheet.write(f'A{i}', label, bold_format)
                summary_sheet.write(f'B{i}', value)
                detail_sheet.write(f'A{i}', label, bold_format)
                detail_sheet.write(f'B{i}', value)
                if has_invoice_exclusions:
                    detail_exclusions_sheet.write(f'A{i}', label, bold_format)
                    detail_exclusions_sheet.write(f'B{i}', value)

            # Adjust the column widths for D & E (Total ISCIs and ISCI Detections)
            summary_sheet.set_column(0, 2, width=40)  # Adjusting column D & E to 20 characters wide
            summary_sheet.set_column(2, 6, width=20)  # Adjusting column D & E to 20 characters wide
            detail_sheet.set_column(3, 6, 20, quantity_format)  # Adjusting column D & E to 20 characters wide


            # Add the 'inv_iscis' value above the detail columns
            # detail_sheet.write('E10', 'Total ISCIs', bold_format)
            # detail_sheet.write('E11', inv_iscis, bold_format)  # Writing the extracted 'inv_iscis' value
            detail_sheet.write(9, 3, 'Advertiser ISCIs', bold_format)  # New header for "Advertiser ISCIs"
            detail_sheet.write(9, 4, 'Total ISCIs', bold_format)

            if inv_detections > 0:
                # Add "ISCI Detections" and "Total Detections" columns to the right of "Total ISCIs"
                detail_sheet.write(9, 5, 'ISCI Detections', bold_format)
                detail_sheet.write(9, 6, 'Total Detections', bold_format)

            # Initialize a variable to track the first occurrence of each advertiser
            advertiser_written = {}

            # Write the custom headers in row 11
            for col_num, column in enumerate(summary_columns):
                custom_header = custom_summary_headers.get(column, column)
                summary_sheet.write(10, col_num, custom_header, bold_format)

            for col_num, column in enumerate(detail_columns):
                custom_header = custom_detail_headers.get(column, column)
                detail_sheet.write(10, col_num, custom_header, bold_format)

            if has_invoice_exclusions:
                for col_num, column in enumerate(detail_exclusion_columns):
                    custom_header = custom_detail_headers_exclusions.get(column, column)
                    detail_exclusions_sheet.write(10, col_num, custom_header, bold_format)


            # Start writing data from row 11
            summary_start_row = 11
            detail_start_row = 11

            # Write summary data
            for row_num, row_data in enumerate(summary_group.itertuples(index=False), start=summary_start_row):
                for col_num, column in enumerate(summary_columns):
                    if column == 'li_total':
                        new_invoice_total += getattr(row_data, column)
                    value = getattr(row_data, column)
                    cell_format = (
                        currency_format if column in currency_columns else
                        quantity_format if column in quantity_columns else
                        date_format if column in date_columns else None
                    )
                    summary_sheet.write(row_num, col_num, value, cell_format)
                sfdc_opp_detail_prep_df = pd.DataFrame(columns=sfdc_opp_detail_columns)
                sfdc_opp_detail_prep_df = summary_group[sfdc_opp_detail_columns].drop_duplicates(subset=['invoice_id','LineDesc']).copy()
                sfdc_opp_detail_df = pd.concat(
                    [sfdc_opp_detail_df, sfdc_opp_detail_prep_df],
                    ignore_index=True
                )

            # Write Grand Total
            grand_total_label_row = summary_start_row + len(summary_group)
            summary_sheet.write(grand_total_label_row, len(summary_columns) - 4, 'Grand Total:', bold_right_format)
            summary_sheet.write(grand_total_label_row, len(summary_columns) - 3, summary_group['li_quantity'].sum(), quantity_bold_format)
            # summary_sheet.write(grand_total_label_row, len(summary_columns) - 1, summary_group['inv_total'].iloc[0], currency_bold_format)
            summary_sheet.write(grand_total_label_row, len(summary_columns) - 1, new_invoice_total, currency_bold_format)

            inv_total_detections = 0
            # Write detail data
            for row_num, row_data in enumerate(detail_group.itertuples(index=False), start=detail_start_row):
                advertiser_name = getattr(row_data, 'sfdc_advertiser_name')
                isci = getattr(row_data, 'isci')
                isci_detections = getattr(row_data, 'count_detections')
                inv_total_detections += isci_detections

                # Get the advertiser ISCI count (adv_iscis) by matching invoice_id and sfdc_advertiser_name

                # advertiser_isci = summary_group[
                #     (summary_group['invoice_id'] == invoice_id) &
                #     (summary_group['sfdc_advertiser_name'] == advertiser_name)
                # ]['count_iscis_by_advertiser'].iloc[0]

                # filtered_series = summary_group[
                #     (summary_group['invoice_id'] == invoice_id) &
                #     (summary_group['sfdc_advertiser_name'] == advertiser_name)
                # ]['count_iscis_by_advertiser']

                # if not filtered_series.empty:
                #     advertiser_isci = filtered_series.iloc[0]
                # else:
                #     advertiser_isci = 0

                # if not summary_group[
                #     (advertiser_summary_group['invoice_id'] == invoice_id) &
                #     (advertiser_summary_group['sfdc_advertiser_name'] == advertiser_name)
                # ].empty else None

                # If this is the first row for this advertiser, write the ISCI count in the "Advertiser ISCIs" column
                if advertiser_name not in advertiser_written:
                    filtered_series = detail_group[
                        (detail_group['invoice_id'] == invoice_id) &
                        (detail_group['sfdc_advertiser_name'] == advertiser_name)
                    ]['count_iscis_by_advertiser']
                    if not filtered_series.empty:
                        advertiser_isci = filtered_series.iloc[0]
                    else:
                        advertiser_isci = 0

                #     advertiser_isci = summary_group[
                #     (summary_group['invoice_id'] == invoice_id) &
                #     (summary_group['sfdc_advertiser_name'] == advertiser_name)
                # ]['count_iscis_by_advertiser'].iloc[0]
                    detail_sheet.write(row_num, 3, advertiser_isci, bold_format)
                    advertiser_written[advertiser_name] = True
                else:
                    detail_sheet.write(row_num, 3, '', bold_format)  # Leave empty for subsequent rows

                # Write the total ISCIs value in the "Total ISCIs" column
                if not total_iscis_inserted:
                    inv_iscis = summary_group[
                    (summary_group['invoice_id'] == invoice_id)
                ]['count_iscis_by_invoice'].iloc[0]
                    detail_sheet.write(row_num, 4, inv_iscis, bold_format)
                    total_iscis_inserted = True

                # if len(detail_detection_counts_group) > 0:
                #     # Get ISCI detections
                #     # isci_detections = len(detail_df[['detections']])
                #     detail_detection_counts_group[
                #         (detail_detection_counts_group['invoice_id'] == invoice_id) &
                #         (detail_detection_counts_group['sfdc_advertiser_name'] == advertiser_name) &
                #         (detail_detection_counts_group['isci'] == isci)
                #     ]['isci_detections'].iloc[0]
                # else:
                #     isci_detections = 0
                #     # if not detail_df[
                #     #     (detail_df['invoice_id'] == invoice_id) &
                #     #     (detail_df['sfdc_advertiser_name'] == advertiser_name) &
                #     #     (detail_df['isci'] == isci)
                #     # ].empty else None

                #     # Write ISCI detections
                detail_sheet.write(row_num, 5, isci_detections if isci_detections else 0, bold_format)
                detail_sheet.write(11, 6, inv_total_detections, bold_format)


                for col_num, column in enumerate(detail_columns):
                    value = getattr(row_data, column)
                    cell_format = (
                        currency_format if column in currency_columns else
                        quantity_format if column in quantity_columns else
                        date_format if column in date_columns else None
                    )
                    detail_sheet.write(row_num, col_num, value, cell_format)

            if has_invoice_exclusions:
                # Write detail data
                for row_num, row_data in enumerate(detail_exclusions_group.itertuples(index=False), start=detail_start_row):
                    for col_num, column in enumerate(detail_exclusion_columns):
                        value = getattr(row_data, column)
                        cell_format = (
                            currency_format if column in currency_columns else
                            date_format if column in date_columns else None
                        )
                        detail_exclusions_sheet.write(row_num, col_num, value, cell_format)

            # Auto-fit columns to content starting from A2
            for col_num, column in enumerate(summary_columns):
                max_length = max(
                    summary_group[column].astype(str).map(len).max(),
                    len(custom_summary_headers.get(column, column))
                ) + 2
                summary_sheet.set_column(col_num, col_num, (max_length + 10))

            for col_num, column in enumerate(detail_columns):
                max_length = max(
                    detail_group[column].astype(str).map(len).max(),
                    len(custom_detail_headers.get(column, column))
                ) + 2
                detail_sheet.set_column(col_num, col_num, (max_length + 10))

        # Upload the file to Google Cloud Storage
        blob_path = f'invoices/{billing_month}/{invoice_target}/{excel_file_name}'
        blob = bucket.blob(blob_path)
        blob.upload_from_filename(excel_file_name)
        blob2_path = f'invoices/{billing_month}/for_csms/{account_csm}/{invoice_target}/{excel_file_name}'
        blob = bucket.blob(blob2_path)
        blob.upload_from_filename(excel_file_name)

        # Generate signed URL for public access
        file_url = generate_signed_url(bucket_name, blob_path, expiration=30)

        # Upsert the record into BigQuery
        upsert_invoice_record(invoice_id, file_url, bcm_index, sfdc_account_id, sfdc_advertiser_id, billing_type)

        # Optionally, delete the local file after upload
        os.remove(excel_file_name)

    print("Monthly billing Excel files generated, uploaded to GCS, and upserted into BigQuery successfully.")


    sfdc_opp_summary_df['invoice_id'] = sfdc_opp_summary_df['invoice_id'].astype(str)
    upload_dataframe_to_bigquery(sfdc_opp_summary_df, sfdc_opp_summary_table_id, project_id)

    sfdc_opp_detail_df_unique = sfdc_opp_detail_df.drop_duplicates().copy()
    sfdc_opp_detail_df_unique['invoice_id'] = sfdc_opp_detail_df_unique['invoice_id'].astype(str)
    sfdc_opp_detail_df_unique['quantity'] = sfdc_opp_detail_df_unique['quantity'].astype(str)
    sfdc_opp_detail_df_unique['price'] = sfdc_opp_detail_df_unique['price'].astype(str)
    sfdc_opp_detail_df_unique['li_total'] = sfdc_opp_detail_df_unique['li_total'].astype(str)
    upload_dataframe_to_bigquery(sfdc_opp_detail_df_unique, sfdc_opp_detail_table_id, project_id)

def initial_task_upload(bcm_index):
    summary_group_sort_by = ['Invoice_ID__c']

    sync_type = "CUSTOM"
    sfdc_task_obj = ['Task']
    sync_salesforce_tables(sync_salesforce_url, sync_salesforce_auth_token, sync_type, sfdc_task_obj)

    sfdc_object_name = 'Task'

    sfdc_external_id_field = 'Invoice_ID__c'

    # sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)


    # Query the summary and detail views. 0053h000007adzPAAQ
    tasks_query = f"""
    SELECT Subject, OwnerId, Status, Broadcast_Month__c, WhatId, Advertiser__c, Invoice_total__c, Draft_Invoice_Preview__c as long_url, short_url, Priority, ActivityDate, Invoice_ID__c , RecordTypeId
    FROM `{project_id}.{dataset_id}._billing_tasks_info` where bcm_index = {bcm_index}
    order by Invoice_ID__c
    """

    task_df = bigquery_client.query(tasks_query).to_dataframe()

    if len(task_df) > 0:

        task_df['short_url'] = task_df.apply(
            lambda row: get_short_url(row['long_url']) if pd.isna(row['short_url']) or row['short_url'] == '' else row['short_url'],
            axis=1
        )



        # Define your BigQuery table ID (project_id.dataset_id.table_id)
        inv_table_id = f'{project_id}.{dataset_id}._billing_invoice_files'

        # Update BigQuery table with new short URLs
        update_bq_invoice_table_short_url(task_df, inv_table_id)

        needs_short_url_query = f"""
        SELECT * FROM `{project_id}.{dataset_id}._billing_invoice_files` where coalesce(excel_path_short, '') = ''
        """

        needs_short_url_df = bigquery_client.query(needs_short_url_query).to_dataframe()
        if len(needs_short_url_df) > 0:
            needs_short_url_df['excel_path_short'] = needs_short_url_df.apply(
                lambda row: get_short_url(row['excel_path']) if pd.isna(row['excel_path_short']) or row['excel_path_short'] == '' else row['excel_path_short'],
                axis=1
            )
            needs_short_url_df['short_url'] = needs_short_url_df['excel_path_short']
            needs_short_url_df['long_url'] = needs_short_url_df['excel_path']
            update_bq_invoice_table_short_url(needs_short_url_df, inv_table_id)

        gbq_to_sfdc_field_map = {
        'Subject': 'Subject',
        'OwnerId': 'OwnerId',
        'Status': 'Status',
        'Broadcast_Month__c': 'Broadcast_Month__c',
        'WhatId': 'WhatId',
        'Advertiser__c': 'Advertiser__c',
        'Invoice_total__c': 'Invoice_total__c',
        'short_url': 'Draft_Invoice_Preview__c',
        'Priority': 'Priority',
        'ActivityDate': 'ActivityDate',
        'Invoice_ID__c': 'Invoice_ID__c',
        'RecordTypeId': 'RecordTypeId'
        }

        if 'sfdc_task_id' not in task_df.columns:
            task_df['sfdc_task_id'] = pd.NaT
        filtered_task_df = pd.DataFrame()
        filtered_task_df = task_df[task_df['sfdc_task_id'].isna()]
        final_task_col = ['Subject', 'OwnerId','Status','Broadcast_Month__c','WhatId','Advertiser__c','Invoice_total__c','short_url','Priority','ActivityDate','Invoice_ID__c', 'RecordTypeId']

        final_task_df = filtered_task_df[final_task_col].copy()

        # Sync Encoder_Group__c object
        if len(final_task_df) > 0:
            response = sync_data_ids_tasks(final_task_df, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)

            sync_salesforce_tables(sync_salesforce_url, sync_salesforce_auth_token, sync_type, sfdc_task_obj)
        else:
            print("No new tasks to sync.")

        excel_short_path_update_sql = f"""
            UPDATE `{project_id}.{dataset_id}._billing_invoice_files` f
            set f.excel_path_short = t.Draft_Invoice_Preview__c
            from `{project_id}.{dataset_id}.sfdc_task_obj` t
            where safe_cast(t.Invoice_ID__c as float64) = safe_cast(f.invoice_id as float64)
            and t.AccountId = sfdc_account_id
            AND coalesce(f.excel_path, '') <> ''
            AND coalesce(f.excel_path_short, '') = '';
        """
        # Execute the query
        query_job = bigquery_client.query(excel_short_path_update_sql)

        # Wait for the query to finish
        query_job.result()

def update_sfdc_opp_url():
    update_sfdc_opp_url_query = f"""
        update `{project_id}.{dataset_id}._export_sfdc_opp_summary` op
        set op.invoice_short_url = b.excel_path_short
        from `{project_id}.{dataset_id}._billing_invoice_files` b
        where op.invoice_id = b.invoice_id
        AND op.invoice_short_url = ''
        AND coalesce(b.excel_path_short, '') <> '';
        """

    # Execute the query
    update_sfdc_opp_url_query_job = bigquery_client.query(update_sfdc_opp_url_query)

    # Wait for the query to finish
    update_sfdc_opp_url_query_job.result()

def update_tasks_needing_adjustment():
    # @title reset the task status of adjusted invoices

    sync_type = "CUSTOM"
    sfdc_task_obj = ['Task']
    sync_salesforce_tables(sync_salesforce_url, sync_salesforce_auth_token, sync_type, sfdc_task_obj)

    needs_adjustment_query = f"""
    SELECT Id as sfdc_task_id FROM `adhoc-billing.avs_billing_process._billing_tasks_not_approved`
    """
    needs_adjustment_df = bigquery_client.query(needs_adjustment_query).to_dataframe()

    for index, row in needs_adjustment_df.iterrows():
        try:
            sf.Task.update(row['sfdc_task_id'], {'Status': 'Review Invoice - adjustments made'})
        except Exception as e:
            print(f"Error while updating Task at index {index}: {e}")

def process_approved_invoices():
    # @title Sync BVS Data to SFDC functions
    # @title process approved invoice tasks to opportunities
    update_sfdc_opp_url_query = f"""
        update `adhoc-billing.avs_billing_process._export_sfdc_opp_summary` op
        set op.invoice_short_url = b.excel_path_short
        from `adhoc-billing.avs_billing_process._billing_invoice_files` b
        where op.invoice_id = b.invoice_id
        AND op.invoice_short_url = ''
        AND coalesce(b.excel_path_short, '') <> '';
        """

    # Execute the query
    update_sfdc_opp_url_query_job = bigquery_client.query(update_sfdc_opp_url_query)

    # Wait for the query to finish
    update_sfdc_opp_url_query_job.result()

    # Function to get PricebookEntryId
    def get_pricebook_entry_id(product_id, pricebook_id):
        query = f"""
        SELECT Id
        FROM PricebookEntry
        WHERE Product2Id = '{product_id}'
        AND Pricebook2Id = '{pricebook_id}'
        AND IsActive = True
        LIMIT 1
        """
        result = sf.query(query)
        if result['totalSize'] > 0:
            return result['records'][0]['Id']
        else:
            raise Exception(f"No active PricebookEntry found for Product2Id {product_id} in Pricebook2Id {pricebook_id}")


    sync_type = "CUSTOM"
    sfdc_task_obj = ['Task']
    sync_salesforce_tables(sync_salesforce_url, sync_salesforce_auth_token, sync_type, sfdc_task_obj)


    approvals_summary_query = f"""
        WITH approvedInvoices AS (
        SELECT cast(Invoice_ID__c as string) as invoice_id from `adhoc-billing.avs_billing_process.sfdc_task_obj`
        WHERE lower(Status) in ('approve to bill')
        AND RecordTypeId = '012UV000000KpzFYAS'
        )
        select ops.* from approvedInvoices a
        join `adhoc-billing.avs_billing_process._export_sfdc_opp_summary` ops
        ON cast(a.invoice_id as float64) = cast(ops.invoice_id as float64)
        where sfdc_opportunity_id = '';
        """


    approvals_summary_df = bigquery_client.query(approvals_summary_query).to_dataframe()

    approvals_detail_query = f"""
        WITH approvedInvoices AS (
        SELECT cast(Invoice_ID__c as string) as invoice_id from `adhoc-billing.avs_billing_process.sfdc_task_obj`
        WHERE lower(Status) in ('approve to bill')
        AND RecordTypeId = '012UV000000KpzFYAS'
        )
        select ops.* from approvedInvoices a
        join `adhoc-billing.avs_billing_process._export_sfdc_opp_detail` ops
        ON cast(a.invoice_id as float64) = cast(ops.invoice_id as float64)
        where sfdc_opportunity_id = '';
        """


    approvals_detail_df = bigquery_client.query(approvals_detail_query).to_dataframe()
    approvals_summary_df['invoice_short_url']
    for col in (approvals_summary_df.columns):
        approvals_summary_df[col] = approvals_summary_df[col].astype(str)

    for col in (approvals_detail_df.columns):
        approvals_detail_df[col] = approvals_detail_df[col].astype(str)

    # Create Opportunities and store their IDs
    # opportunity_id_map = {}

    # approvals_summary_df.columns.to_list()
    # approvals_summary_df['RecordTypeId'].drop_duplicates()
    try:
        blisspoint_invoice_group = approvals_summary_df[approvals_summary_df['sfdc_account_id'] == '0013h00001Vc5S1AAJ']
        blisspoint_invoice_id = blisspoint_invoice_group['invoice_id'].iloc[0]
    except:
        blisspoint_invoice_id = ''

    for index, row in approvals_summary_df.iterrows():

        try:
            if row['sfdc_account_id'] == '0013h00001Vc5S1AAJ':
                response = sf.Opportunity.create({'Billing_Start_Date__c': row['billing_start_date'],'Invoice_Summary_Preview2__c': row['invoice_short_url'], 'Invoice_ID__c':row['invoice_id'], 'Name': row['op_name'], 'OwnerId': opp_owner_id, 'Type': opp_type, 'StageName': opp_stage_name, 'CloseDate': row['closeDate'], 'AccountId': row['sfdc_account_id'], 'BVS_Sales_Rep__c': '', 'BVS_Invoice_Terms__c': row['BVSInvoiceTerms'], 'RecordTypeId': opp_record_type_id, 'Pricebook2Id': opp_pricebook_id, 'Order_Date__c': row['orderdate'], 'PO_Number__c': ''})
            elif row['sfdc_advertiser_id'] == '':
                response = sf.Opportunity.create({'Billing_Start_Date__c': row['billing_start_date'],'Invoice_Summary_Preview2__c': row['invoice_short_url'], 'Invoice_ID__c':row['invoice_id'], 'Name': row['op_name'], 'OwnerId': opp_owner_id, 'Type': opp_type, 'StageName': opp_stage_name, 'CloseDate': row['closeDate'], 'AccountId': row['sfdc_account_id'], 'BVS_Sales_Rep__c': '', 'BVS_Invoice_Terms__c': row['BVSInvoiceTerms'], 'RecordTypeId': opp_record_type_id, 'Pricebook2Id': opp_pricebook_id, 'Order_Date__c': row['orderdate'], 'PO_Number__c': ''})
            else:
                response = sf.Opportunity.create({'Billing_Start_Date__c': row['billing_start_date'],'Invoice_Summary_Preview2__c': row['invoice_short_url'], 'Invoice_ID__c':row['invoice_id'], 'Name': row['op_name'], 'OwnerId': opp_owner_id, 'Type': opp_type, 'StageName': opp_stage_name, 'CloseDate': row['closeDate'], 'AccountId': row['sfdc_account_id'], 'BVS_Sales_Rep__c': '', 'BVS_Invoice_Terms__c': row['BVSInvoiceTerms'], 'RecordTypeId': opp_record_type_id, 'Pricebook2Id': opp_pricebook_id, 'Order_Date__c': row['orderdate'], 'PO_Number__c': '', 'Advertiser_Object__c': row['sfdc_advertiser_id']})
            print(response)
            stage_update_response = sf.Opportunity.update(response['id'], {'StageName': opp_stage_name, 'Invoice_Summary_Preview2__c': row['invoice_short_url']})
            print(stage_update_response)
            # Save the new Opportunity ID to the dataframe
            approvals_summary_df.at[index, 'sfdc_opportunity_id'] = response['id']
        except Exception as e:
            print(f"Error while uploading Opportunity at index {index}: {e}")

    print("Uploaded data to Salesforce Opportunities and retrieved new IDs")
    print(approvals_summary_df)

    sfdc_op_ids_to_invoice = approvals_summary_df[['invoice_id', 'sfdc_opportunity_id']].drop_duplicates().copy()
    sfdc_op_ids_to_invoice.reset_index(drop=True, inplace=True)

    # Create a mapping from 'invoice_id' to 'new_value'
    mapping = sfdc_op_ids_to_invoice.set_index('invoice_id')['sfdc_opportunity_id']
    approvals_detail_df['sfdc_opportunity_id'] = approvals_detail_df['invoice_id'].map(mapping).fillna(approvals_detail_df['sfdc_opportunity_id'])

    for index, row in approvals_detail_df.iterrows():
        try:
            # Create the Opportunity Line Item
            if row['invoice_id'] == blisspoint_invoice_id:
                response = sf.OpportunityLineItem.create({
                    'OpportunityId': row['sfdc_opportunity_id'],
                    'Product2Id': row['Product2Id'],
                    # 'ProductCode': row['ProductCode'],
                    'Description': row['LineDesc'],
                    'Quantity': row['quantity'],
                    'TotalPrice': row['li_total'],  # Or 'TotalPrice': row['TotalPrice']
                    # Include any other optional fields as needed
                })
                print(f"Opportunity Line Item created with ID: {response['id']}")
            else:
                response = sf.OpportunityLineItem.create({
                    'OpportunityId': row['sfdc_opportunity_id'],
                    'Product2Id': row['Product2Id'],
                    # 'ProductCode': row['ProductCode'],
                    'Description': row['LineDesc'],
                    'Quantity': row['quantity'],
                    'UnitPrice': row['price'],  # Or 'TotalPrice': row['TotalPrice']
                    # Include any other optional fields as needed
                })
                print(f"Opportunity Line Item created with ID: {response['id']}")
        except Exception as e:
            print(f"Error while uploading Opportunity Line Item at index {index}: {e}")

    processed_opps_query = f"""
        SELECT Id, Invoice_Id__c as invoice_id from `adhoc-billing.avs_billing_process.sfdc_task_obj`
        WHERE lower(Status) in ('approve to bill')
        AND RecordTypeId = '012UV000000KpzFYAS'
        """
    processed_opps_df = bigquery_client.query(processed_opps_query).to_dataframe()
    processed_opps_df['sfdc_opportunity_id'] = processed_opps_df['invoice_id'].map(mapping).fillna(approvals_detail_df['sfdc_opportunity_id'])

    for index, row in processed_opps_df.iterrows():
        try:
            sf.Task.update(row['Id'], {'Status': 'Opportunity Created'})
        except Exception as e:
            print(f"Error while updating Task at index {index}: {e}")

    table_1 = f'{project_id}.{dataset_id}._export_sfdc_opp_detail'
    table_2 = f'{project_id}.{dataset_id}._export_sfdc_opp_summary'

    update_bq_sfdc_opportunity_id(sfdc_op_ids_to_invoice, table_1)
    update_bq_sfdc_opportunity_id(sfdc_op_ids_to_invoice, table_2)

    sfdc_objs = ["Task","Opportunity","OpportunityLineItem"]
    sync_salesforce_tables(sync_salesforce_url, sync_salesforce_auth_token, sync_type, sfdc_objs)



In [ ]:
# @title Sync BVS Data to SFDC functions
# Define the GBQ query and field mapping

def sync_bvs_customers_to_sfdc():

    sfdc_object_name = 'BVS_Customer__c'


    sfdc_external_id_field = 'customer_id__c'

    sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)
    print(sfdc_max_updated)


    gbq_query = f"""
        SELECT customer_id, account_id, contract_item, customer_name, contract_number, sales_person_code, deleted, last_audit_id, last_updated
        FROM `bigquery-sandbox-393916.prod_avs.customers`
        WHERE customer_id in (select customer_id from `{project_id}.{dataset_id}.new_or_updated_customer`)
    """

    gbq_to_sfdc_field_map = {
        'customer_id': 'customer_id__c',
        'account_id': 'account_id__c',
        'contract_item': 'Contract_Item__c',
        'customer_name': 'Name',
        'contract_number': 'Contract_Number__c',
        'sales_person_code': 'Sales_Person_Code__c',
        'deleted': 'Is_Deleted__c',
        'last_audit_id': 'Last_Audit_ID__c',
        'last_updated': 'Last_Updated__c'
    }

    # Sync BVS_Customer__c object
    response = sync_data_ids(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)
    if response:
        custom_sync_objs.append(response)
    else:
        print("Sync completed.")

def sync_bvs_profiles_to_sfdc():
    sfdc_object_name = 'BVS_Profile__c'

    sfdc_external_id_field = 'BVS_Profile_ID__c'

    sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)


    gbq_query = f"""
        SELECT profile_id, profile_name, deleted, default_asset_code, last_audit_id, last_updated, ARRAY_TO_STRING(
                ARRAY(
                    SELECT TO_JSON_STRING(attr)
                    FROM UNNEST(attributes) AS attr
                ), ', '
            ) AS attributes
        FROM `bigquery-sandbox-393916.prod_avs.profiles`
        WHERE profile_id in (select profile_id from `{project_id}.{dataset_id}.new_or_updated_profile`)
    """
    gbq_to_sfdc_field_map = {
        'profile_id': 'BVS_Profile_ID__c',
        'profile_name': 'Name',
        'deleted': 'Is_Deleted__c',
        'default_asset_code': 'Default_Asset_Code__c',
        'last_audit_id': 'Last_Audit_ID__c',
        'last_updated': 'Last_Updated__c',
        'attributes': 'Attributes__c'
    }

    # Sync BVS_Customer__c object
    response = sync_data_ids(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)
    if response:
        custom_sync_objs.append(response)
    else:
        print("Sync completed.")

    # upadte the bvs_customer_id lookup object in sfdc
    print("Sync completed.")

def sync_bvs_formats_to_sfdc():
    sfdc_object_name = 'BVS_Format__c'

    sfdc_external_id_field = 'BVS_Format_ID__c'

    sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)

    # Define the GBQ query and field mapping
    gbq_query = f"""
        SELECT format_id, profile_id, customer_id, format_name, report_breakup, deleted, last_audit_id, last_updated
        FROM `bigquery-sandbox-393916.prod_avs.formats`
        WHERE format_id in (select format_id from `{project_id}.{dataset_id}.new_or_updated_format`)
    """
    gbq_to_sfdc_field_map = {
        'format_id': 'BVS_Format_ID__c',
        'profile_id': 'Profile_ID__c',
        'customer_id': 'Customer_ID__c',
        'format_name': 'Name',
        'report_breakup': 'report_breakup__c',
        'deleted': 'Is_Deleted__c',
        'last_audit_id': 'Last_Audit_ID__c',
        'last_updated': 'Last_Updated__c'
    }

    # Sync BVS_Customer__c object
    response = sync_data_ids(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)
    if response:
        custom_sync_objs.append(response)

    update_bvs_customers_lookup_on_format(sf)

    update_bvs_profile_lookup_on_format(sf)

    print("Sync completed.")

def sync_bvs_encoder_groups_to_sfdc():
    sfdc_object_name = 'Encoder_Group__c'

    sfdc_external_id_field = 'Encoder_Group_ID__c'

    sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)


    gbq_query = f"""
        SELECT encoder_group_id, encoder_group_name, deleted, last_audit_id, last_updated
        FROM `bigquery-sandbox-393916.prod_avs.encoder_groups`
        WHERE encoder_group_id in (select encoder_group_id from `{project_id}.{dataset_id}.new_or_updated_encoder_group`)
    """


    gbq_to_sfdc_field_map = {
        'encoder_group_id': 'Encoder_Group_ID__c',
        'encoder_group_name': 'Name',
        'deleted': 'Is_Deleted__c',
        'last_audit_id': 'Last_Audit_ID__c',
        'last_updated': 'Last_Updated__c'
    }


    # Sync Encoder_Group__c object
    response = sync_data_ids(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)
    if response:
        custom_sync_objs.append(response)
    # upadte the bvs_customer_id lookup object in sfdc
    print("Sync completed.")

def sync_bvs_encoder_devices_to_sfdc():
    sfdc_object_name = 'Encoder_Device__c'

    sfdc_external_id_field = 'bvs_encoder_device_id__c'

    # sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)


    gbq_query = f"""
        SELECT encoder_id, encoder_name, encoder_group_id, deleted, last_audit_id, last_updated
        FROM `bigquery-sandbox-393916.prod_avs.encoders`
        WHERE encoder_id in (select encoder_id from `{project_id}.{dataset_id}.new_or_updated_encoder`)

    """

    gbq_to_sfdc_field_map = {
        'encoder_id': 'bvs_encoder_device_id__c',
        'encoder_name': 'BVS_Encoder_Name__c',
        'encoder_group_id': 'bvs_encoder_group_id__c',
        'deleted': 'Is_Deleted__c',
        'last_audit_id': 'Last_Audit_ID__c',
        'last_updated': 'Last_Updated__c'
    }

    # Sync Encoder_Group__c object
    response = sync_data_ids(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)
    if response:
        custom_sync_objs.append(response)

    # upadte the bvs_customer_id lookup object in sfdc
    print("Sync completed.")

def sync_bvs_encoding_advertiserss_to_sfdc():
    sfdc_object_name = 'Encoding_Advertiser__c'

    sfdc_external_id_field = 'Name'

    # sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)


    gbq_query = f"""
    SELECT sfdc_customer_id, bvs_encoding_advertiser_id, advertiser
        FROM `{project_id}.{dataset_id}.encodings_advertisers`

        where trim(bvs_encoding_advertiser_id) not in (
            select trim(Name) from `{project_id}.{dataset_id}.sfdc_encoding_advertiser__c_obj`
        )
        GROUP BY sfdc_customer_id, bvs_encoding_advertiser_id, advertiser

    """



    gbq_to_sfdc_field_map = {
        'bvs_encoding_advertiser_id': 'Name',
        'sfdc_customer_id': 'BVS_Customer__c',
        'advertiser': 'Encoding_Advertiser__c',

    }


    # Sync Encoding_Advertiser__c object
    response = sync_data_with_precalculated_id(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)
    if response:
        custom_sync_objs.append(response)

    # upadte the bvs_customer_id lookup object in sfdc
    print("Sync completed.")

def sync_bvs_encoding_product_codes_to_sfdc():
    sfdc_object_name = 'Encoding_Product_Code__c'

    sfdc_external_id_field = 'Name'

    # sfdc_max_updated = fetch_max_last_updated(sfdc_object_name)


    gbq_query = f"""
        SELECT sfdc_customer_id, bvs_encoding_product_code_id, product_code
        FROM `{project_id}.{dataset_id}.encodings_product_codes`
        where trim(bvs_encoding_product_code_id) not in
        (select trim(Name) from `{project_id}.{dataset_id}.sfdc_encoding_product_code__c_obj`)
        GROUP BY sfdc_customer_id, bvs_encoding_product_code_id, product_code
    """
    gbq_to_sfdc_field_map = {
        'bvs_encoding_product_code_id': 'Name',
        'sfdc_customer_id': 'BVS_Customer__c',
        'product_code': 'Encoding_Product_Code__c',

    }


    # Sync Encoding_Product_Code__c object
    response = sync_data_with_precalculated_id(gbq_query, gbq_to_sfdc_field_map, sfdc_object_name, sfdc_external_id_field)
    if response:
        custom_sync_objs.append(response)
    # upadte the bvs_customer_id lookup object in sfdc
    print("Sync completed.")



In [ ]:
# @title Data Prep Process Control

if sync_salesforce_with_bvs_before_processing:
    sync_salesforce_tables(sync_salesforce_url, sync_salesforce_auth_token, sync_salesforce_type)
    custom_sync_objs = []

if sync_bvs_with_sfdc_before_processing:
    sync_bvs_customers_to_sfdc()
    sync_bvs_profiles_to_sfdc()
    sync_bvs_formats_to_sfdc()
    sync_bvs_encoder_groups_to_sfdc()
    sync_bvs_encoder_devices_to_sfdc()
    sync_bvs_encoding_advertiserss_to_sfdc()
    sync_bvs_encoding_product_codes_to_sfdc()
    print("Full sync completed.")
else:
    print("Sync skipped.")

if sync_bvs_with_sfdc_before_processing:
    sync_type = "FULL"
    sync_salesforce_tables(sync_salesforce_url, sync_salesforce_auth_token, sync_type, custom_sync_objs)

if rebuild_encodings:
    update_encodings_base()
    get_unmatched_advertisers()
    get_unmatched_accounts()
else:
    print("Skipping encodings base update")

for bcm_index in bcm_index_list:
    print(f"Processing bcm_index: {bcm_index}")
    if rebuild_detection_archive:
        load_detections_into_archive(bcm_index)
    else:
        print("Skipping detection archive update")

bucket_name = storage_bucket_name
bucket = storage_client.bucket(bucket_name)


sfdc_opp_summary_columns = ['invoice_id', 'sfdc_opportunity_id', 'broadcast_month_name', 'sfdc_account_id', 'sfdc_advertiser_id', 'op_name', 'ownerId', 'closeDate', 'poNumber', 'stageName', 'orderdate', 'billing_start_date', 'billing_end_date', 'type', 'BVSInvoiceTerms', 'SOSalesRep', 'PriceBook2Id', 'RecordTypeId', 'invoice_short_url']
sfdc_opp_detail_columns = ['invoice_id',  'sfdc_opportunity_id', 'broadcast_month_name', 'LineDesc', 'Product2Id', 'ProductCode', 'quantity', 'price', 'li_total', 'reporting_cust_name', 'reporting_adv_name']

sfdc_opp_summary_table_name = '_export_sfdc_opp_summary'  # Replace with your desired table name
sfdc_opp_summary_table_id = f'{project_id}.{dataset_id}.{sfdc_opp_summary_table_name}'

sfdc_opp_detail_table_name = '_export_sfdc_opp_detail'  # Replace with your desired table name
sfdc_opp_detail_table_id = f'{project_id}.{dataset_id}.{sfdc_opp_detail_table_name}'
today = datetime.now().date()

# Extract year and month from bcm_index
year = int(bcm_index)
# Multiply fractional part by 100 and round to get the month
month = int(round((bcm_index - year) * 100))
bcm_index_str = f"{year}-{month:02d}"
first_month_date = f"{bcm_index_str}-01"
billing_start_date = first_month_date
last_day = calendar.monthrange(year, month)[1]
billing_end_date = f"{bcm_index_str}-{last_day:02d}"

if gen_encoder_excel:
    generate_encoder_excel(bcm_index)
else:
    print("Skipping encoder excel generation")

if gen_weekly_excel:
    generate_weekly_excel(bcm_index)
else:
    print("Skipping weekly excel generation")

if gen_monthly_excel:
    generate_monthly_excel(bcm_index)
else:
    print("Skipping monthly excel generation")

if generate_sfdc_tasks:
    initial_task_upload(bcm_index)
    update_sfdc_opp_url()
else:
    print("Skipping sfdc task generation")

if process_sfdc_opps_from_invoices:
    update_tasks_needing_adjustment()
    process_approved_invoices()
else:
    print("Skipping sfdc opp processing")



In [ ]:
get_unmatched_advertisers()
